# <center> Process simulation V.23.04.13 (Stable)
## <center>Vaegan 4.0

# To do

1. Handle Station Overload with wingmen
2. Sheet filling
3. Shuffling AI agent



### Importing dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output as cl
import csv
import cv2
import math
import colorama
from colorama import Fore

printflag = False
plot = 1

### Hyper Parameters

In [ ]:
scaling_factor = 1
#Design parameter
total_bikes = 360
no_of_stations = 20 # total number of stations
station_capacity = 100# station_capacity

# Electricity
no_of_days = 30 #no of days
rupees_per_unit = 6.75
#Service and Travel time
operation_start_time = 8 # service start time -- 8am
operation_end_time = 23 #service end time --- 8 pm
closing_time = 165 #time of closing
shuffle_closing_time = 45
#station priority
high_priority_traffic_range = [50,100]
low_priority_traffic_range = [0,50]
medium_priority_traffic_range = [30,50]
# time probablity
peak_time = [(8,10.5), (16,20)]
peak_time_probablity = [70,100]
off_peak_time_probablity = [0,35]

#plot orientation
row = 5
column = 4
ppts = 60 # plot per time stamp


#shuffling parameters
shuffling = True
min_h_high = 6
max_h_high = 12
min_h_low = 2
max_h_low = 3
#Wingman parameters
no_of_wm = 30 * scaling_factor

### Classes for city, station, bike Expense, Account, Billing, Income

In [ ]:
class city:
    def __init__(self):
        self.established_stations = []
        self.running_bikes = []
        self.initial_outward_status_list = []
        self.shuffle_counter = 0
        self.ride_counter = 0
        self.losing_customer_percentage = 0
        self.shuffling_failure_counter = 0
        self.wingman_shuffle_failure_counter = 0
        self.bike_shuffle_failure_counter = 0
        self.failplot = []
        self.printer = []
        self.booking_log = []
        self.whole_booking_log = []
        self.kmc =[]
        self.user_km_counter = []
        self.shuffle_km_counter = []
        self.user_idle_time_counter = []
        self.user_overall_time_counter = []
        self.total_kms_user_ridden = 0
        self.total_shuffle_kms_ridden = 0
        self.total_idle_time = 0
        self.total_kms_user_ridden_this_day = 0
        self.total_shuffle_kms_ridden_this_day = 0
        self.total_idle_time_this_day = 0
        self.shuffle_ready_stations = []
        self.shuffle_requesting_stations = []
        self.clock=0
        self.Day = 0
        self.Wingman = []
        self.history_objects = dict()
        self.shuffle_logger = []
        self.min_h_shuffle_logger = []
        self.max_h_shuffle_logger = []
        self.wingman_calling_log = dict()
        self.call_counter = 0
    def establish_new_station(self,st_name,s_id):
        st_obj = station(st_name,station_id = s_id)
        if printflag:print(f"New station {st_name} has been established")
        return st_obj
    def purchase_bike(self,bikeid,initial_battery = 0):
        bike_obj = bike(bikeid)
        bike_obj.battery_percentage = initial_battery
        if printflag:print(f"Purchace of bike {bikeid} is suscessful")
        return bike_obj
    def reset_day(self):
        self.clock=0
        self.shuffle_counter = 0
        self.ride_counter =0
        self.losing_customer_percentage = 0
        self.shuffling_failure_counter = 0
        self.booking_log =[]
        self.failplot = []
        self.printer = []
        self.kallapetti = []
        self.kmc = []
        self.bike_shuffle_failure_counter = 0
        self.wingman_shuffle_failure_counter = 0
        self.call_counter =0
        self.user_km_counter = []
        self.shuffle_km_counter = []
        self.user_idle_time_counter = []
        self.user_overall_time_counter = []
        self.total_kms_user_ridden = 0
        self.total_shuffle_kms_ridden = 0
        self.total_idle_time = 0
        self.total_kms_user_ridden_this_day = 0
        self.total_shuffle_kms_ridden_this_day = 0
        self.total_idle_time_this_day = 0
        for station_ in self.established_stations:
            station_.customer_count = 0
            station_.charging_time = 0
            station_.ready_bike_count = len(station_.current_available_bike)
            station_.shuffle_requested =0
            station_.shuffle_donated = 0
            station_.shuffle_index = 0
            self.failplot = []
            station_.ride_request_list = []
            station_.ride_request_list_mean =[]
            station_.ready_bike_count_list = []
            station_.ready_bike_count_list_mean = []
            station_.ride_request_list_whole =[]
            self.history =[]
            #station_.over_night_charge()
        for bike in self.running_bikes:
            bike.ride_counter = 0
            bike.running_time = 0
            bike.charging_time = 0
            bike.shuffling = False
            bike.shuttle_timer = 0
            bike.shuffle_counter = 0
            bike.use_time = 0
            bike.km = 0
            bike.km_counter = 0
            bike.active = False
            bike.charging =False
            bike.use = False
            bike.shuffling = False
            bike.total_customer_km = 0
            bike.total_shuffler_km = 0
            bike.battery_percentage = 0
            #bike.active = False
        for wm in self.Wingman:
            wm.engaged = False
            wm.recovery = False
            wm.recovery_time = 0
            wm.recovery_time_i = 0
            wm.call_km = 0
        self.test_wm=[]
    def book_ride(self,bike,orgin,start_time,travel_time,use_time,km,day,destination):
        bike.book_index = self.ride_counter
        bike.destination = destination
        bike.active = True
        bike.charging = False
        bike.ride_start_time = start_time
        bike.travel_time = travel_time
        bike.km_log+= km
        #print("ride booked")
        bike.ride_counter +=1
        orgin.customer_count +=1
        Timestart = Time_con(bike.ride_start_time)
        self.booking_log.append({'Ride id': f"Book{bike.book_index}",'Booking_Time': f"Day {day} {Timestart}",'Booking_Time_mins':bike.ride_start_time, 'Orgin':orgin.station_id+str('--')+ orgin.station_name,'Bike': bike.bike_id,'User':'Customer', 'Intial_battery_percentage': bike.battery_percentage })
        bike.ready = False
        orgin.current_available_bike.remove(bike)
        # assigning class variable with passed arguments
        bike.use_time = use_time
        bike.use = True
        bike.km = km
        bike.total_customer_km +=km
        bike.km_counter = 0
        bike.initial_battery = bike.battery_percentage
    def customer_request(self):
        demand_dict = dict()
        for station in coimbatore.established_stations:
            station.traffic = station.outward_traffic_status
            random = np.random.randint(0,100)
            mean = 0
            sd = 1 * scaling_factor
            if (random < station.traffic):
                sample = np.random.normal(mean,sd)
                sample = round(sample)
                if (sample < 0): sample*=-1
                no_of_customer = sample
            else:
                no_of_customer = 0
            #print(f"Returing {no_of_customer} for {station.station_name}")
            demand_dict.update({station.station_id:no_of_customer})

        return demand_dict

    def end_ride (self,bike_obj,day,time):
        destination = bike_obj.destination
        orgin = bike_obj.station
        #bill = round(coimbatore.ride_charge_calculation(bike_obj),2)
        #self.kallapetti.append(bill)
        TimeEnd = Time_con(time)


        self.history_objects.update({f"Book{bike_obj.book_index}":{"orgin":orgin,"Destination":destination,"Bike": bike_obj}})

        for book in reversed(coimbatore.booking_log):
            if(book["Ride id"] == f"Book{bike_obj.book_index}"):
                book["Ending_Time"] = TimeEnd
                book["Ending_Time_mins"] = time
                book["Destination"] = destination.station_id+str('--') + destination.station_name
                book["Kms_travelled"] = bike_obj.km_counter
                book["Travel_time"] = bike_obj.travel_time
                book["Final_Battery_percentage"] = bike_obj.battery_percentage
                book["Battery_depleted"] = (bike_obj.initial_battery - bike_obj.battery_percentage)
                book["Usetime"] = bike_obj.use_time
                Billing.method_1(book)
                #book["Bill"] = bill
                orgin.history.append({"Start":book})
                destination.history.append({"End":book})
                break
        self.user_idle_time_counter.append(bike_obj.travel_time - bike_obj.use_time)
        self.user_overall_time_counter.append(bike_obj.travel_time + bike_obj.use_time)
        bike_obj.station_history_obj.append((orgin,destination))
        bike_obj.active = False
        bike_obj.ride_start_time =0
        bike_obj.travel_time = 0
        bike_obj.charging = True
        bike_obj.use = False
        self.kmc.append(bike_obj.km_counter)
        self.user_km_counter.append(bike_obj.km_counter)
        bike_obj.km_counter = 0
        #orgin.current_available_bike.remove(bike_obj)
        destination.current_available_bike.append(bike_obj)
        bike_obj.station =   destination
        #bike_obj.ready = True


    def ride_charge_calculation(self,bike):
        km_charge = bike.km_counter * 0.3375
        time_charge = 0.0675 * (bike.travel_time - bike.use_time)
        extra_charge = 0
        tax = 0
        margin = 0
        total_charge = km_charge+time_charge+extra_charge+tax+margin
        return total_charge

    def over_night_partial_charge(self):
        for bike in self.running_bikes:
            while bike.battery_percentage < 100:
                #print ("\r", round(bike.battery_percentage,2))
                bike.battery_percentage += 0.27777777778 # 1/(6*60)
                bike.sai_list.append(0.27777777778)
                bike.charging_time +=1
                bike.station.charging_time +=1
                bike.active = False
            bike.charging = False


    def shuffle_ride_terminate (self,bike,day,time):
        wingman = bike.wm
        #wingman.recovery = True
        destination = bike.shuffle_destination
        wingman.location = destination
        wingman.engaged = False
        wingman.location.current_available_wingman.append(wingman)
        orgin = bike.station
        bike.active = False
        bike.shuttle_timer += bike.travel_time
        orgin.shuffle_donated+=1
        self.kmc.append(bike.km_counter)
        self.shuffle_km_counter.append(bike.km_counter)
        destination.shuffle_requested+=1
        wingman.shuffling_time+= bike.travel_time
        wingman.km += bike.km
        wingman.shuffling_counter+=1
        #print(bike.travel_time)
        destination.current_available_bike.append(bike)
        bike.station = destination
        bike.shuffling = False
        bike.ready = True
        bike.charging = True
        #stop using when the

        bike.use = False
        bill = round(coimbatore.ride_charge_calculation(bike),2)
        TimeEnd = Time_con(time)


        for book in reversed(coimbatore.booking_log):
            if(book["Ride id"] == f"Book{bike.book_index}"):
                book["Ending_Time"] = TimeEnd
                book["Ending_Time_mins"] = time
                book["Destination"] = destination.station_id+str('--') + destination.station_name
                book["Kms_travelled"] = bike.km_counter
                book["Travel_time"] = bike.travel_time
                book["Final_Battery_percentage"] = bike.battery_percentage
                book["Battery_depleted"] = (bike.initial_battery - bike.battery_percentage)
                book["Usetime"] = bike.use_time
                book["Bill"] = bill
                orgin.history.append({"Start":book})
                destination.history.append({"End":book})
                break
        for book in reversed(wingman.history):
            if(book["Ride id"] == f"Book{bike.book_index}"):
                book["Ending_Time"] = TimeEnd
                book["Ending_Time_mins"] = time
                book["Destination"] = destination.station_id+str('--') + destination.station_name
                book["Kms_travelled"] = bike.km_counter
                book["Travel_time"] = bike.travel_time
                book["Final_Battery_percentage"] = bike.battery_percentage
                book["Battery_depleted"] = (bike.initial_battery - bike.battery_percentage)
                book["Usetime"] = bike.use_time
                book["Bill"] = bill
                book["Recovery time"] = wingman.recovery_time
                break

        hist = {f"Book{bike.book_index}":{"orgin":orgin,"Destination":destination,"Bike": bike_obj}}
        self.history_objects.update({f"Book{bike.book_index}":{"orgin":orgin,"Destination":destination,"Bike": bike}})
        wingman.history_obj.append(hist)
        bike.travel_time = 0
        bike.ride_start_time =0
        bike.km_counter = 0
        self.wingman_recovery(wingman)

    def shuffle_ride_initiate(self,bike,orgin,dest,travel_time,current_time,day,wingman):
        bike.book_index = self.ride_counter
        bike.km = km = distance_matrix[int(orgin.station_index)][int(dest.station_index)]
        bike.km_counter = 0
        bike.km_log+= km
        bike.active = True
        bike.charging = False
        wingman.recovery = False

        bike.ride_start_time = current_time
        wingman.ride_start_time = current_time
        bike.travel_time = travel_time#Time_chart[int(current_time//60)][int(orgin.station_index)][int(dest.station_index)]
        coimbatore.shuffle_counter+=1
        bike.ready = False
        bike.shuffle_counter +=1
        bike.shuffling = True
        bike.wm = wingman
        wingman.location = orgin
        wingman.engaged = True
        wingman.location.current_available_wingman.remove(wingman)
        Timestart = Time_con(bike.ride_start_time)
        self.booking_log.append({'Ride id': f"Book{bike.book_index}",'Booking_Time': f"Day {day} {Timestart}"  ,'Booking_Time_mins':bike.ride_start_time, 'Orgin':orgin.station_id+str('--')+ orgin.station_name,'Bike': bike.bike_id,'User': wingman.employee_id,'Intial_battery_percentage': bike.battery_percentage})
        wingman.history.append({'Ride id': f"Book{bike.book_index}",'Booking_Time': f"Day {day} {Timestart}",'Booking_Time_mins':bike.ride_start_time, 'Orgin':orgin.station_id+str('--')+ orgin.station_name,'Bike': bike.bike_id,'User': wingman.employee_id,'Intial_battery_percentage': bike.battery_percentage })
        #No idle time for shuffling
        bike.use_time = bike.travel_time
        # shuffling random km from 0 to 5
        bike.total_shuffler_km +=bike.km
        bike.use = True
        bike.initial_battery = bike.battery_percentage
        orgin.current_available_bike.remove(bike)
        orgin.shuffle_ready_bikes.remove(bike)
        orgin.ready_bike_list.remove(bike)
        orgin.ready_bike_count -= 1
        bike.shuffle_destination = dest


    def insert_bike(self,station_,bike_):
        self.running_bikes.append(bike_)
        if (len(station_.current_available_bike)< station_.capacity):
            station_.current_available_bike.append(bike_)
            if printflag:print(f"{bike_.bike_id} is added to station {station_.station_name} ")
            bike_.station = station_
        else:
            print("Station capacity reached !!")

    def call_wingman(self, destination, time):
        assigned = False
        for i in destination.sorted_travel_time_chart:
            if len(i[3].current_available_wingman) !=0:
                ready_wingman = i[3].current_available_wingman[0]
                assigned = True
                break
        if assigned:
            #print(ready_wingman.employee_id, "s")
            current_time = time
            ready_wingman.engaged = True
            ready_wingman.available = False
            ready_wingman.arriving_station = destination
            #print(ready_wingman.location.station_id,"a",ready_wingman.employee_id)
            origin = ready_wingman.location
            #print(origin.station_id,"b",ready_wingman.employee_id)
            travel_time = origin.travel_time_chart[ready_wingman.arriving_station.station_index][1]
            ready_wingman.eta = current_time+travel_time
            origin.current_available_wingman.remove(ready_wingman)
            ready_wingman.ride_start_time = coimbatore.clock
            #print(f"{ready_wingman.employee_id} removed from {origin.station_id}")
            ready_wingman.calling_log_station_holding_annotations = (len(origin.current_available_bike),len(destination.current_available_bike),f"Call{self.call_counter:03d}")
            self.call_counter +=1
            ready_wingman.call = True


    def close_wingman_call(self, wm):
        if wm.call:
            self.wingman_calling_log.update({wm.calling_log_station_holding_annotations[2]:{"Calling Time":Time_con(wm.ride_start_time),"Arriving time":Time_con(coimbatore.clock),"Wingman_id":wm.employee_id, "Departing station": wm.location.station_id + " (" +wm.location.station_name+")"+ f"Before: {wm.calling_log_station_holding_annotations[0]} After: {len(wm.location.current_available_bike)}","Arriving station": wm.arriving_station.station_id + " (" +wm.arriving_station.station_name+")"+ f"Before: {wm.calling_log_station_holding_annotations[1]} After: {len(wm.arriving_station.current_available_bike)}","station objects":(wm.location,wm.ride_start_time,wm.arriving_station,coimbatore.clock) }})
            wm.call_km +=distance_matrix[int(wm.location.station_index)][int(wm.arriving_station.station_index)]
        wm.call = False

        ready_wingman = wm
        ready_wingman.engaged = False
        ready_wingman.available = True
        ready_wingman.location = ready_wingman.arriving_station

        ready_wingman.location.current_available_wingman.append(ready_wingman)
        #print(f"{ready_wingman.employee_id} added to {ready_wingman.location.station_id} and status {ready_wingman.engaged} existing_eta of {ready_wingman.eta}")
        ready_wingman.eta = 0

    def wingman_recovery(self, wm):
        recovery_needed = True
        origin = wm.location
        dest = origin.sorted_arriving_stations[0]
        wm.arriving_station = dest
        Travel_time = Time_chart[int(coimbatore.clock//60)][dest.station_index][origin.station_index]

        if origin == wm.next_station :
            recovery_needed = False

        if recovery_needed:
            wm.eta = coimbatore.clock+Travel_time
            wm.engaged = True
            wm.available = False
            origin.current_available_wingman.remove(wm)


#____________________________________________________________________________________________________________________________
class station:
    def __init__(self,station_name,capacity = station_capacity, station_id = "VS - 100"):
        self.station_id = station_id
        #self.weight = 0
        self.current_available_bike = []
        self.capacity = capacity
        self.station_name = station_name
        self.initial_outward_status = ''
        self.min_h = 0
        self.max_h = 0
        self.customer_count = 0
        self.charging_time = 0
        self.ride_request_list = []
        self.ride_request_mean = 0
        self.ready_bike_count = 0
        self.shuffle_index = 0
        self.shuffle_ready_bikes = []
        self.travel_time_chart = []
        self.sorted_travel_time_chart = []
        self.requesting_station = False
        self.traffic = 0
        #self.shuffle_timer = 0
        self.shuffle_requested = 0
        self.shuffle_donated =0
        self.ready_bike_list = []
        self.ready_bike_count_list = []
        self.ready_bike_count_list_mean =[]
        self.latitude = 0
        self.longitude = 0
        self.ride_request_list_mean =[]
        self.ride_request_list_whole =[]
        self.inward_traffic_status = ''
        self.outward_trafiic_status = ''
        self.station_index = 0
        self.current_available_wingman = []
        self.history = []
        self.sorted_arriving_stations = []
'''def over_night_charge(self):
        for bike in self.current_available_bike:
            bike.battery_percentage = 1
            bike.active = False
            bike.charging_time += (6*60)
            self.charging_time+= (6*60)'''

#____________________________________________________________________________________________________________________________
class bike:
    def __init__(self, bike_id):

        self.battery_percentage = 0
        self.bike_id = bike_id
        self.active= False
        self.charging = False
        self.charge_start_time = 0
        self.ride_start_time = 0
        self.travel_time = 0
        self.station = 0
        self.ride_counter = 0
        self.running_time= 0
        self.charging_time = 0
        self.ready = True
        self.shuffle_counter = 0
        self.shuffle_destination = ''
        self.shuffling = False
        self.shuttle_timer= 0
        self.use_time = 0
        self.use = False
        self.km = 0
        self.km_counter = 0
        self.km_log =0
        self.total_customer_km = 0
        self.total_shuffler_km = 0
        self.initial_battery = 0
        self.sai_list = []
        self.wm =0
        self.station_history_obj =[]
    def charge_vehicle(self,start_time):
        self.charging = True
        self.charge_start_time = start_time
        self.active= False
#____________________________________________________________________________________________________________________________

class Wingman:
    def __init__(self, Employee_id):
        self.employee_id = Employee_id
        self.engaged = False
        self.shuffling_time = 0
        self.wallet_list = []
        self.wallet_sum = []
        self.history = []
        self.history_obj =[]
        self.km = 0
        self.shuffling_counter = 0
        self.recovery = False
        self.available = True
        self.arriving_station = ""
        self.eta = 0
        self.recovery_time = 0
        self.recovery_time_i = 0
        self.travel_time = 0
        self.location =""
        self.next_station = ""
        self.calling_log_station_holding_annotations = ""
        self.call = False
        self.call_km = 0


class expenses:
    def __init__(self):

        # ASSUMPTIONS and FACTS for RIDE CHARGES CALCULATION
        # 1. A bike with 100% charge can go 80kms & a 100% charging requires 4units & 1unit is ₹6.75 (A&F)
        # 2. Each station is assumed to be approx. 1000sq.ft and rent per month would be ₹35000 (A&F)
        # 3. Work hours suggested by Indian Govt. is 8hrs/day => 48hrs/week (F)
        # 4. Work days suggested by Indian Govt. is 6days/week => 26days/month(30days) (A&F)

        range_of_a_bike = 80#kms
        current_consumption_for_a_full_charge = 4#units(kWh)
        charges_per_unit = 6.75 # INR

        self.per_full_charge = current_consumption_for_a_full_charge * charges_per_unit # = 4*6.75
        self.per_km_ridden = self.per_full_charge / range_of_a_bike # = ₹0.3375
        self.per_shuffle_km_ridden = self.per_km_ridden # = ₹0.3375

        rent_per_station_per_month = 35000 # INR
        rent_for_all_stations_per_month = rent_per_station_per_month * no_of_stations # = ₹7,00,000
        self.rent_for_all_stations_per_day = rent_for_all_stations_per_month / no_of_days # = ₹23,333.3333334

        individual_salary_per_month_8h = 20000 # INR | which is for 8hrs/day for 26days
        individual_salary_per_day_8h = individual_salary_per_month_8h / 26#days | as the ₹20000 salary is for 26days
        individual_salary_per_hour = individual_salary_per_day_8h / 8#hrs
        total_salary_per_hour = individual_salary_per_hour * no_of_stations # = ₹1923
        billable_hours_per_day = 15#hrs
        self.total_personnel_salary_per_day = total_salary_per_hour * billable_hours_per_day # = ₹28845

        self.total_auxiliary_electricity_charges_per_day = 3474.685 # INR | Refer Notion for table of contents

        self.total_network_charges_per_day = 549.88 # INR | Refer Notion for table of contents

        total_bike_maintenance_charges_per_day = 5322.60 # INR | Refer Notion for table of contents
        total_station_maintenance_charges_per_day = 4000 # INR | Refer Notion for table of contents
        self.total_maintenance_charges_per_day = total_bike_maintenance_charges_per_day + total_station_maintenance_charges_per_day

        self.overall_per_km_expense = 0
        self.for_overall_wingman_payment_this_day = 0
        self.overall_per_km_expense_daily = []
        self.overall_per_km_expense_so_far = 0
        self.for_overall_wingman_payment_so_far = 0
        self.for_all_stations_rent_so_far = 0
        self.for_total_personnel_salary_so_far = 0
        self.for_total_auxiliary_electricity_usage_so_far = 0
        self.for_total_network_usage_so_far = 0
        self.for_total_maintenance_so_far = 0

    def per_km(self):
        self.for_total_km_ridden_this_day = self.per_km_ridden * coimbatore.total_kms_user_ridden_this_day
        self.for_total_km_ridden_so_far = self.per_km_ridden * coimbatore.total_kms_user_ridden

    """def for_idle_time(self):
        coimbatore.total_idle_time = sum(coimbatore.user_idle_time_counter)
        self.for_total_idle_time = self.per_min_of_idle_time * total_idle_time"""

    def per_shuffle_km(self):
        self.for_total_suffle_km_ridden_this_day = self.per_shuffle_km_ridden * coimbatore.total_shuffle_kms_ridden_this_day
        self.for_total_suffle_km_ridden_so_far = self.per_shuffle_km_ridden * coimbatore.total_shuffle_kms_ridden

    def for_wingman_payment(self):
        #self.per_km_of_shuffling = Accounts.income_due_to_shuffling / coimbatore.total_shuffle_kms_ridden
        self.for_overall_wingman_payment_this_day = np.mean(Accounts.temp_bill_book) * coimbatore.shuffle_counter
        self.for_overall_wingman_payment_so_far+=self.for_overall_wingman_payment_this_day
        #pass

    def for_station_rent(self):
        self.for_all_stations_rent = self.rent_for_all_stations_per_day
        self.for_all_stations_rent_so_far+=self.for_all_stations_rent

    def for_personnel_salary(self):
        self.for_total_personnel_salary = self.total_personnel_salary_per_day
        self.for_total_personnel_salary_so_far+=self.for_total_personnel_salary

    def for_auxiliary_electricity_usage(self):
        self.for_total_auxiliary_electricity_usage = self.total_auxiliary_electricity_charges_per_day
        self.for_total_auxiliary_electricity_usage_so_far+=self.for_total_auxiliary_electricity_usage

    def for_network_usage(self):
        self.for_total_network_usage = self.total_network_charges_per_day
        self.for_total_network_usage_so_far+=self.for_total_network_usage

    def for_maintenance(self):
        self.for_total_maintenance = self.total_maintenance_charges_per_day
        self.for_total_maintenance_so_far+=self.for_total_maintenance

    def for_debt_repayment(self):
        #self.for_total_debt_repayment = self.total_debt_repayment_per_day * Day
        pass

    def for_operations(self):
        self.for_whole_operations = self.for_total_km_ridden_this_day+self.for_total_suffle_km_ridden_this_day+self.for_overall_wingman_payment_this_day+self.for_all_stations_rent+self.for_total_personnel_salary+self.for_total_auxiliary_electricity_usage+self.for_total_network_usage+self.for_total_maintenance
        self.for_whole_operations_so_far = self.for_total_km_ridden_so_far+self.for_total_suffle_km_ridden_so_far+self.for_overall_wingman_payment_so_far+self.for_all_stations_rent_so_far+self.for_total_personnel_salary_so_far+self.for_total_auxiliary_electricity_usage_so_far+self.for_total_network_usage_so_far+self.for_total_maintenance_so_far

    def total(self):
        self.total_expense = self.for_whole_operations #+ self.for_total_debt_repayment
        self.total_expense_so_far = self.for_whole_operations_so_far

    def calculate(self):
        self.per_km()
        #self.for_idle_time()
        self.per_shuffle_km()
        #self.for_wingman_payment()
        self.for_station_rent()
        self.for_personnel_salary()
        self.for_auxiliary_electricity_usage()
        self.for_network_usage()
        self.for_maintenance()
        self.for_debt_repayment()
        self.for_operations()
        self.total()
        self.overall_per_km_expense = self.total_expense / coimbatore.total_kms_user_ridden_this_day
        self.overall_per_km_expense_daily.append(self.overall_per_km_expense)

    def re_calculate(self):
        self.for_operations()
        self.total()
        self.overall_per_km_expense = self.total_expense / coimbatore.total_kms_user_ridden_this_day
        self.overall_per_km_expense_so_far = np.mean(self.overall_per_km_expense)

    def show_day_expenses(self):
        print("\n\n------------------- Expenses this day -------------------------\n")
        print("Km Expenses               : ₹", self.for_total_km_ridden_this_day)
        #print("idle time expenses", self.per_min_of_idle_time)
        print("Shuffle km Expenses       : ₹", self.for_total_suffle_km_ridden_this_day)
        print("Wingman payment expenses  : ₹", self.for_overall_wingman_payment_this_day)
        print("Rent Expenses             : ₹", self.for_all_stations_rent)
        print("Salary Expenses           : ₹", self.for_total_personnel_salary)
        print("Aux. electricity Expenses : ₹", self.for_total_auxiliary_electricity_usage)
        print("Network Expenses          : ₹", self.for_total_network_usage)
        print("Maintenance Expenses      : ₹", self.for_total_maintenance)
        #print("Debt repayment Expenses   : ₹", self.self.for_total_debt_repayment)
        print("Operational Expenses      : ₹", self.for_whole_operations)
        print("\nTotal Expenses            : ₹", self.total_expense)
        print("\nOverall Expenses per km users ride : ₹", self.overall_per_km_expense)
        #print("\n------------------------------------------------------------")

    def show_expenses_so_far(self):
        print("\n\n--------------------- Expenses so far --------------------------\n")
        print("Km Expenses               : ₹", self.for_total_km_ridden_so_far)
        #print("idle time expenses", self.per_min_of_idle_time)
        print("Shuffle km Expenses       : ₹", self.for_total_suffle_km_ridden_so_far)
        print("Wingman payment expenses  : ₹", self.for_overall_wingman_payment_so_far)
        print("Rent Expenses             : ₹", self.for_all_stations_rent_so_far)
        print("Salary Expenses           : ₹", self.for_total_personnel_salary_so_far)
        print("Aux. electricity Expenses : ₹", self.for_total_auxiliary_electricity_usage_so_far)
        print("Network Expenses          : ₹", self.for_total_network_usage_so_far)
        print("Maintenance Expenses      : ₹", self.for_total_maintenance_so_far)

        #print("Debt repayment Expenses   : ₹", self.self.for_total_debt_repayment)
        print("Operational Expenses      : ₹", self.for_whole_operations_so_far)
        print("\nTotal Expenses            : ₹", self.total_expense_so_far)
        print("\nOverall Expenses per km users ride : ₹", self.overall_per_km_expense_so_far)
        print("\nIdle_time_loss_per_min : ₹", Accounts.idle_time_charges)
        #print("\n------------------------------------------------------------")

Expense = expenses()

class accounts:
    def __init__(self):
        self.temp_bill_book = []
        self.bill_book = []
        self.idle_time_charges_list = []
        self.idle_time_charges = 0

    def calculate_km_charges(self):
        self.per_km_charge = Expense.overall_per_km_expense
        self.temp_bill_book = []
        for book in coimbatore.booking_log:
            if book["User"] == "Customer":
                user_km = book["Kms_travelled"]
                km_ridden_charges = user_km * self.per_km_charge
                book["Projected_Bill"] = km_ridden_charges
                self.temp_bill_book.append(km_ridden_charges)
                #print(book["Projected_Bill"], km_ridden_charges, user_km, self.per_km_charge)

    def add_wingman_payment_charges_and_margin(self):
        customer_count = coimbatore.ride_counter
        self.income_due_to_shuffling = np.mean(self.temp_bill_book) * coimbatore.shuffle_counter
        Expense.for_wingman_payment()
        self.wingman_charges_per_customer = Expense.for_overall_wingman_payment_this_day / customer_count
        self.margin = 0
        self.temp_bill_book = []
        for book in coimbatore.booking_log:
            if book["User"] == "Customer":
                book["Projected_Bill"]+=(self.wingman_charges_per_customer + self.margin)
                self.temp_bill_book.append(book["Projected_Bill"])

    def add_idle_time_charges_and_taxes(self):
        loss_for_every_min_of_idle_time = np.mean(self.temp_bill_book) / np.mean(coimbatore.user_overall_time_counter)
        self.temp_bill_book = []
        self.charges_per_min_of_idle_time = loss_for_every_min_of_idle_time
        self.idle_time_charges_list.append(self.charges_per_min_of_idle_time)
        cgst = 0.09
        sgst = 0.09
        self.tax_percentage = cgst+sgst
        for book in coimbatore.booking_log:
            if (book["User"] == "Customer") and ((book["Travel_time"] - book["Usetime"])>0):
                idle_time = book["Travel_time"] - book["Usetime"]
                idle_time_charges = idle_time * self.charges_per_min_of_idle_time
                book["Projected_Bill"]+=idle_time_charges

        for book in coimbatore.booking_log:
            if book["User"] == "Customer":
                book["Projected_Bill"]+=(book["Projected_Bill"] * self.tax_percentage)
                self.temp_bill_book.append(book["Projected_Bill"])

    def generate_projected_bill(self):
        self.calculate_km_charges()
        self.add_wingman_payment_charges_and_margin()
        self.add_idle_time_charges_and_taxes()
        self.idle_time_charges = np.mean(self.idle_time_charges_list)
        self.bill_book += self.temp_bill_book


    def calculate_profit_this_day(self):
        self.gross_profit_this_day = Income.gross_income - Expense.total_expense
        self.net_profit_this_day = Income.net_income - Expense.total_expense

    def calculate_profit_so_far(self):
        self.gross_profit_so_far = Income.gross_income_so_far - Expense.total_expense_so_far
        self.net_profit_so_far = Income.net_income_so_far - Expense.total_expense_so_far

    def show_profit_this_day(self):
        print("\n\n---------------- PROFIT / LOSS (this day) ----------------------\n")
        if(self.gross_profit_this_day) > 0:
            print("GROSS PROFIT THIS DAY : ₹", Fore.GREEN , self.gross_profit_this_day, Fore.RESET)
        if(self.gross_profit_this_day) < 0:
            print("GROSS PROFIT THIS DAY : ₹", Fore.RED , self.gross_profit_this_day, Fore.RESET)

        if(self.net_profit_this_day) > 0:
            print("NET PROFIT THIS DAY   : ₹", Fore.GREEN , self.net_profit_this_day, Fore.RESET)
        if(self.net_profit_this_day) < 0:
            print("NET PROFIT THIS DAY   : ₹", Fore.RED , self.net_profit_this_day, Fore.RESET)
        #print("\n----------------------------------------------------------------------------------")

    def show_profit_so_far(self):
        print("\n\n----------------- PROFIT / LOSS (so far) -----------------------\n")
        if(self.gross_profit_so_far) > 0:
            print("GROSS PROFIT SO FAR   : ₹", Fore.GREEN , self.gross_profit_so_far, Fore.RESET)
        if(self.gross_profit_so_far) < 0:
            print("GROSS PROFIT SO FAR   : ₹", Fore.RED , self.gross_profit_so_far, Fore.RESET)

        if(self.net_profit_so_far) > 0:
            print("NET PROFIT SO FAR     : ₹", Fore.GREEN , self.net_profit_so_far, Fore.RESET)
        if(self.net_profit_so_far) < 0:
            print("NET PROFIT SO FAR     : ₹", Fore.RED , self.net_profit_so_far, Fore.RESET)
        #print("\n----------------------------------------------------------------------------------")

Accounts = accounts()

class billing:
    def __init__(self):
        self.set_per_km_charge = 5
        self.set_idle_time_charge = 0.8
        self.sgst = 0.09
        self.cgst = 0.09

    def method_1(self, book):
        self.for_kms_ridden = book["Kms_travelled"] * self.set_per_km_charge
        book["KM CHARGES"] = self.for_kms_ridden
        self.idle_time = book["Travel_time"] - book["Usetime"]
        self.for_idle_time = self.idle_time * self.set_idle_time_charge
        book["TIME CHARGES"] = self.for_idle_time
        self.for_sgst = (self.for_kms_ridden + self.idle_time) * self.sgst
        self.for_cgst = (self.for_kms_ridden + self.idle_time) * self.cgst
        self.taxes_inclusion = self.for_sgst + self.for_cgst
        book["TAXES"] = self.taxes_inclusion
        self.total_bill = self.for_kms_ridden + self.idle_time + self.taxes_inclusion
        book["TOTAL CHARGES"] = self.total_bill

    def method_2(self):
        pass

Billing = billing()

class income:
    def __init__(self):
        self.gross_income = 0
        self.tax_recovered = 0
        self.net_income = 0
        self.gross_income_list = []
        self.gross_income_so_far = 0
        self.tax_recovered_so_far = 0
        self.net_income_so_far = 0
        self.gross_income_list_so_far = []

    def calculate(self):
        self.gross_income = 0
        self.tax_recovered = 0
        self.net_income = 0
        self.gross_income_list = []
        for book in coimbatore.booking_log:
            if book["User"] == "Customer":
                self.gross_income_list.append(book["TOTAL CHARGES"])
                self.gross_income += book["TOTAL CHARGES"]
                self.tax_recovered += book["TAXES"]
                self.net_income += book["TOTAL CHARGES"] - book["TAXES"]

                self.gross_income_list_so_far.append(book["TOTAL CHARGES"])
                self.gross_income_so_far += book["TOTAL CHARGES"]
                self.tax_recovered_so_far += book["TAXES"]
                self.net_income_so_far += book["TOTAL CHARGES"] - book["TAXES"]


    def show_income_this_day(self):
        print("\n--------------------- INCOME THIS DAY --------------------------\n")
        print(f"GROSS INCOME  : ₹ {self.gross_income}")
        print(f"TAX RECOVERED : ₹ {self.tax_recovered}")
        print(f"NET INCOME    : ₹ {self.net_income}")

    def show_income_so_far(self):
        print("\n---------------------- INCOME SO FAR ---------------------------\n")
        print(f"GROSS INCOME  : ₹ {self.gross_income_so_far}")
        print(f"TAX RECOVERED : ₹ {self.tax_recovered_so_far}")
        print(f"NET INCOME    : ₹ {self.net_income_so_far}")

Income = income()


### Anticipated Station spots

In [ ]:
coimbatore_map = [""]*20
coimbatore_planned_spots = random.sample(coimbatore_map,no_of_stations)

### Creating a city and establishing station planned stations

In [ ]:
coimbatore = city()
vs_counter = 1
for places in coimbatore_planned_spots:
    new_station = coimbatore.establish_new_station(places, f"VS - {vs_counter:02d}")
    coimbatore.established_stations.append(new_station)
    vs_counter+=1


### Reading Map

In [ ]:
coordinates =[]
with open('Station_coordinate.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for c,row_ in enumerate(reader):
        if(c!=0):
            coordinates.append((float(row_[-2]),float(row_[-1]), row_[1]))
for c, station in enumerate(coimbatore.established_stations):
    station.latitude = coordinates[c][0]
    station.longitude = coordinates[c][1]
    station.station_name = coordinates[c][2]
    station.station_index = c

### Reading Traffic chart

In [ ]:
with open('inward.csv', 'r') as f:
    reader = csv.reader(f)
    inward = {}
    for row_ in reader:

        inward[row_[0]] = [float(i) for i in row_[1:]]
    inward.pop('', None)
with open('outward.csv', 'r') as f:
    reader = csv.reader(f)
    outward = {}
    for row_ in reader:
        outward[row_[0]] = [float(i) for i in row_[1:]]
    outward.pop('', None)

distance_matrix = np.round(np.load("Distance_matrix.npy"))
Time_chart = np.load("Travel-Time-chart.npy")

for c,station in enumerate(coimbatore.established_stations):
    station.outward_traffic_status = outward[station.station_id][0]
    station.inward_traffic_status = inward[station.station_id][0]
mean_f = []
for station in coimbatore.established_stations:
    mean_f.append(outward[station.station_id][0])
mean_f = np.mean(mean_f)


### Purchasing and filling bikes in each station and Assigning initial traffic

In [ ]:
bike_id_counter = 1


for vehicle in range(total_bikes):
    for count, station in enumerate(coimbatore.established_stations):
        station.initial_outward_status = outward[station.station_id][0]
        coimbatore.initial_outward_status_list.append(outward[station.station_id][0])
        if len(coimbatore.running_bikes)<total_bikes:

            if station.initial_outward_status < mean_f:

                #for i in range(1):
                #purchasing bike
                bike_obj = coimbatore.purchase_bike("V{:04d}".format(bike_id_counter),initial_battery = 0 )
                bike_id_counter +=1
                # adding bike to station
                coimbatore.insert_bike(coimbatore.established_stations[count],bike_obj)

            elif station.initial_outward_status > mean_f and len(coimbatore.running_bikes)<total_bikes-1:
                for i in range(2):
                    #purchasing bike
                    bike_obj = coimbatore.purchase_bike("V{:04d}".format(bike_id_counter),initial_battery = 0 )
                    bike_id_counter +=1
                    # adding bike to station
                    coimbatore.insert_bike(coimbatore.established_stations[count],bike_obj)

            else:
                #purchasing bike
                bike_obj = coimbatore.purchase_bike("V{:04d}".format(bike_id_counter),initial_battery = 0 )
                bike_id_counter +=1
                # adding bike to station
                coimbatore.insert_bike(coimbatore.established_stations[count],bike_obj)

#Test Print. Uncomment the below line to troubleshoot "coimbatore.running_bikes"
#print(len(coimbatore.running_bikes))
#print(coimbatore.initial_outward_status_list)# = []

### Recruiting  Wingmen

In [ ]:
for i in range(no_of_wm):
    Wingman_ = Wingman(f"Wingman{i+1:03d}")
    Wingman_.location = coimbatore.established_stations[np.random.randint(0,19)]
    Wingman_.location.current_available_wingman.append(Wingman_)
    coimbatore.Wingman.append(Wingman_)

### Plot function

In [ ]:
def Time_con(time):return f"{(int(time)//60)+operation_start_time :02d}:{int(time)%60 :02d} "
def plot_stat(time):
    index = []
    for i in range(row):
        for j in range(column):
            index.append((i,j))

    fig, axs = plt.subplots(row,column,figsize=(15, 15))
    fig.suptitle(Time_con(time), fontsize=30)
    for s in range(len(coimbatore.established_stations)):

        battery = []
        bike_ids = []
        niram = []
        for Veh in coimbatore.established_stations[s].current_available_bike:
            battery.append(Veh.battery_percentage)
            bike_ids.append(Veh.bike_id)
            if Veh.active : niram.append('g')
            elif Veh.charging: niram.append('r')
            else : niram.append('k')
        xlab = [' ']*  coimbatore.established_stations[s].capacity
        y = [0]*  coimbatore.established_stations[s].capacity
        y[:len(battery)] = battery
        xlab[:len(bike_ids)] = bike_ids
        x = np.linspace(0,coimbatore.established_stations[s].capacity -1 ,coimbatore.established_stations[s].capacity)
        #print(x,y)
        #y = y[:coimbatore.established_stations[s].capacity]
        axs[index[s]].bar(x,y, color = niram)
        axs[index[s]].set_title(f"{coimbatore.established_stations[s].station_id} \nOutward: {coimbatore.established_stations[s].outward_traffic_status} \nInward: {coimbatore.established_stations[s].inward_traffic_status}  ")
        axs[index[s]].set_xticks(x,xlab,rotation=90)
        axs[index[s]].set_ylim(0,100)
        #plt.show()
    fig.tight_layout()
    cl(wait = True)
    plt.show()

def print_day_vehicle_report():
    total_ride_counter = 0
    total_running_time = 0
    total_charging_time = 0
    print ("        Vehicle Report       ",'\n')
    print("|Vehicle_id| no of rides |  running_time | charging_time |")
    print('_________________________________________________________')
    for bike in coimbatore.running_bikes:
        print(f"|  {bike.bike_id}   |      {bike.ride_counter:02d}      |      {bike.running_time}      |     {bike.charging_time}      |")
        total_ride_counter += bike.ride_counter
        total_running_time += bike.running_time
        total_charging_time += bike.charging_time
    print('_________________________________________________________')
    print(f"|   Total  |     {total_ride_counter}     |      {total_running_time}     |    {total_charging_time}     |")
    print("\n\n")
def print_station_one_day_report(only_value = False):
    print ("        Station Report       ",'\n')
    Total_charging_time  = 0
    Total_customer_count = 0
    stations_charging_log = []

    if not only_value:print(f"| Station |Transactions|Charging time|Station name")
    if not only_value:print("__________________________________________________")
    for count, station in enumerate(coimbatore.established_stations) :
        stations_charging_log.append(station.charging_time)
        if not only_value:print(f"|Station {count+1}|     {station.customer_count}     |    {station.charging_time}     |{station.station_name}")
        Total_charging_time+=station.charging_time
        Total_customer_count +=station.customer_count
    if not only_value:print("___________________________________________________")
    if not only_value:print(f"|  Total  |     {Total_customer_count}    |    {Total_charging_time}    |")
    if not only_value:print("\n\n")
    if not only_value:print (f"Maximum charges per station per month INR {round(((max(stations_charging_log) /60 )*(4/6))*rupees_per_unit,2)}" )
    if not only_value:print (f"Average charges per station per month INR {round(((np.mean(stations_charging_log) /60 )*(4/6))*rupees_per_unit,2)}" )
    if not only_value:print("\n")
    #print(stations_charging_log,sum(stations_charging_log))
    return round(((np.mean(stations_charging_log) /60 )*(4/6))*rupees_per_unit,2)

    #print(f"Total expected income {}")
def plot_report(ref):
    plt.figure(figsize=(15, 5))
    label = []
    for i in range(operation_start_time,operation_end_time):
        label += [f'{i}']+['']*59
    label [-1] = 20
    timestamp = np.linspace(0,((operation_end_time- operation_start_time)*60)-1,(operation_end_time- operation_start_time)*60)
    for i in np.array(ref).T:


        plt.plot(i)
        plt.xticks(timestamp, label)
        plt.xticks(fontsize=12)
        plt.xticks(rotation=90)
        plt.axhline(y = 0., color = 'r', linestyle = '-')
        plt.axhline(y = 25, color = 'y', linestyle = '-',linewidth = 5)
    plt.title("Battery of all vehicles over time")
    plt.xlabel('time')
    plt.ylabel('Battery level')
    plt.show()
def plot_active_vehicle_in_each_station(ref):
    label = np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)+1)
    fig, axs = plt.subplots(no_of_stations, 1,sharex = False ,figsize=(25, 150))
    fig.suptitle("Available bike at each station at end of every hour timeseries", fontsize = 30)
    tit = [i.station_name for i in coimbatore.established_stations]
    timestamp = np.linspace(0,((operation_end_time- operation_start_time)*60)-1,(operation_end_time- operation_start_time)*60)
    for i in range (len(coimbatore.established_stations)):
        axs[i].plot(label[:-1],(np.array(ref).T[i]))
        axs[i].set_title(f'{tit[i]}',fontsize=20)
        axs[i].set_xticks(label,label.astype(int))
    yticks = range(0,np.max(np.array(ready_state))+1,5)
    for ax in axs:
        ax.set_yticks(yticks)
    plt.show()

def plot_available_plot(ready_state):
    index = []
    for i in range(row):
        for j in range(column):
            index.append((i,j))
    label = np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)+1)

    fig, axs = plt.subplots(row,column,figsize=(20, 20))
    fig.suptitle("Available bike at each station at end of every hour", fontsize = 30)
    fig.text(0.5, 0.04, 'Time', ha='center',fontsize = 20)
    fig.text(0.04, 0.5, 'Available bikes', va='center', rotation='vertical',fontsize = 20)
    for count,i in enumerate(np.array(ready_state).T):
        axs[index[count]].bar(np.linspace(operation_start_time,operation_end_time,operation_end_time - operation_start_time),i)
        axs[index[count]].set_xticks(label,label.astype(int))
        axs[index[count]].set_title(f"{coimbatore.established_stations[count].station_name}")
        axs[index[count]].set_ylim(0,np.max(np.array(ready_state))+1)
    plt.show()
def plot_vehicles_accumulation(max_vehicle):
    for i in range(no_of_stations):
        #plt.plot(np.array(max_vehicle).T[i])
        print (f"Maximum Accumulated vehicles at {coimbatore.established_stations[i].station_id} --- {max(np.array(max_vehicle).T[i])}, {int((np.argmax(np.array(max_vehicle).T[i]))/60)+8}:{(np.argmax(np.array(max_vehicle).T[i]))%60}")
    print('____________________________________________________________________________________________________________________')
    for i in range(no_of_stations):
        print (f"Minimum Accumulated vehicles at {coimbatore.established_stations[i].station_id} --- {min(np.array(max_vehicle).T[i])}, {int((np.argmin(np.array(max_vehicle).T[i]))/60)+8}:{(np.argmin(np.array(max_vehicle).T[i]))%60}")
        #plt.show()


def plot_booking_status(book_counter):
    plt.bar(np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)),book_counter)
    plt.title ("Ride booked for every hour")
    label = np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)+1)
    plt.xticks(label,label.astype(int))
    plt.xlabel("Time")
    plt.ylabel ("Bookings")
    plt.show()

def plot_in_ride_status(in_ride_bikes):
    chunk_size = 60
    averages = []
    for i in range(0, len(in_ride_bikes), chunk_size):
        chunk = in_ride_bikes[i:i+chunk_size]
        avg = sum(chunk) / len(chunk)
        averages.append(avg)
    plt.bar(np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)),averages)
    plt.title ("Average in ride vehicles for every hour")
    label = np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)+1)
    plt.xticks(label,label.astype(int))
    plt.xlabel("Time")
    plt.ylabel ("no of vehicles")
    plt.show()


def plot_customer_lose(losing_customer):
    top_average = []
    print("Total Lost customer: ", np.sum(losing_customer))
    for lose in np.array(losing_customer).T:
        chunk_size = 60
        averages = []
        #print(lose.shape)
        for i in range(0, len(lose), chunk_size):
            chunk = lose[i:i+chunk_size]
            avg = sum(chunk) / len(chunk)
            averages.append(avg)
        top_average.append(averages)
    top_average = list(np.array(top_average).T)
    index = []
    for i in range(row):
        for j in range(column):
            index.append((i,j))
    label = np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)+1)

    fig, axs = plt.subplots(row,column,figsize=(20, 20))
    fig.suptitle("Average losing customer for every hour", fontsize = 30)
    fig.text(0.5, 0.04, 'Time', ha='center',fontsize = 20)
    fig.text(0.04, 0.5, 'Losing customer', va='center', rotation='vertical',fontsize = 20)
    for count,i in enumerate(np.array(top_average).T):
        axs[index[count]].bar(np.linspace(operation_start_time,operation_end_time-1,operation_end_time-operation_start_time),i)
        axs[index[count]].set_xticks(label,label.astype(int))
        axs[index[count]].set_title(f"{coimbatore.established_stations[count].station_name}")
        axs[index[count]].set_ylim(0,np.max(np.array(top_average))+1)
    plt.show()

def plot_customer_gain(gaining_customer):
    top_average = []
    print("Total Gained customer: ", np.sum(gaining_customer))
    for lose in np.array(gaining_customer).T:
        chunk_size = 60
        averages = []
        #print(lose.shape)
        for i in range(0, len(lose), chunk_size):
            chunk = lose[i:i+chunk_size]
            avg = sum(chunk) / len(chunk)
            averages.append(avg)
        top_average.append(averages)
    top_average = list(np.array(top_average).T)
    index = []
    for i in range(row):
        for j in range(column):
            index.append((i,j))
    label = np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)+1)

    fig, axs = plt.subplots(row,column,figsize=(20, 20))
    fig.suptitle("Average gaining customer for every hour", fontsize = 30)
    fig.text(0.5, 0.04, 'Time', ha='center',fontsize = 20)
    fig.text(0.04, 0.5, 'Gaining customer', va='center', rotation='vertical',fontsize = 20)
    for count,i in enumerate(np.array(top_average).T):
        axs[index[count]].bar(np.linspace(operation_start_time,operation_end_time-1,operation_end_time-operation_start_time),i)
        axs[index[count]].set_xticks(label,label.astype(int))
        axs[index[count]].set_title(f"{coimbatore.established_stations[count].station_name}")
        axs[index[count]].set_ylim(0,np.max(np.array(top_average))+1)
    plt.show()
def print_lose_data_report():
    total_running_time=0
    total_ride_counter = 0
    for bike in coimbatore.running_bikes:
        total_running_time += bike.running_time
        total_ride_counter += bike.ride_counter
    timer = 0
    for bike in coimbatore.running_bikes:
        timer+=bike.shuttle_timer
    print (f"Total number of rides booked in this day {coimbatore.ride_counter} ")
    print(f"Total Customer Transactions: {total_ride_counter} ")
    print(f"Total Shuffle Transactions: {coimbatore.shuffle_counter} ")
    print(f"Minutes of Shuffling: {timer} minutes")
    print(f"Minutes of Running: {total_running_time} minutes")
    print(f"Percentage of customer lost in this day {coimbatore.losing_customer_percentage} %")
    print(f"Failed shuffling {coimbatore.shuffling_failure_counter}")
    print(f"Failed shuffling by lack of wingmen: {coimbatore.wingman_shuffle_failure_counter}")
    print(f"Failed shuffling by lack of bikes: {coimbatore.bike_shuffle_failure_counter}")
    print('\n')
    print("Station_ ID   |      Donated      |   Kedachadhu    |")
    print('_________________________________________________________________')
    for station in coimbatore.established_stations:
        print(f"{station.station_id}       |        {station.shuffle_donated :02d}         |      {station.shuffle_requested :02d}        |")
    print('_________________________________________________________________')
    shuttle_fail_plot__()
"""def plot_shuffle_fail_():
    in_ride_bikes = [i*100 for i in coimbatore.failplot]
    chunk_size = 60
    averages = []
    for i in range(0, len(in_ride_bikes), chunk_size):
        chunk = in_ride_bikes[i:i+chunk_size]
        avg = sum(chunk) / len(chunk)
        averages.append(avg)
    plt.bar(np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)),averages)
    plt.title ("Average failure in shuffling for every hour")
    label = np.linspace(operation_start_time,operation_end_time,(operation_end_time - operation_start_time)+1)
    plt.xticks(label,label.astype(int))
    plt.xlabel("Time")
    plt.ylabel ("Failure_rate")
    plt.show()  """
def shuttle_fail_plot__():
    label =[]
    for i in range(operation_start_time,operation_end_time):
        label += [f'{i}']+['']*59
    label [-1] = 20
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18, 5),sharex = True)
    timestamp = np.linspace(0,((operation_end_time- operation_start_time)*60)-1,(operation_end_time- operation_start_time)*60)
    axes[0].plot(coimbatore.bike_fail)
    axes[1].plot(coimbatore.wingman_fail)
    axes[0].set_ylim(-1,10)
    axes[1].set_ylim(-1,10)
    axes[0].text(len(timestamp)/2, 9, 'Failure by Bikes unavailablity', horizontalalignment='center', verticalalignment='center')
    axes[1].text(len(timestamp)/2, 9, 'Failure by Wingman unavailablity', horizontalalignment='center', verticalalignment='center')
    plt.xticks(timestamp, label)
    plt.show()
def ready_vs_request_plot(compare_bool = False,s = 0):
    global_counter = 0
    label = []
    for i in range(operation_start_time,operation_end_time):
        label += [f'{i}']+['']*59
    label [-1] = 20
    compare,compare_ = coimbatore.established_stations[s].ride_request_list_whole+coimbatore.established_stations[s].ride_request_list,coimbatore.established_stations[s].ready_bike_count_list

    timestamp = np.linspace(0,((operation_end_time- operation_start_time)*60)-1,(operation_end_time- operation_start_time)*60)
    for station in coimbatore.established_stations:
        plt.figure(figsize = (30,5))
        plt.xticks(timestamp, label)
        plt.xticks(fontsize=30)
        plt.xticks(rotation=90)
        plt.title(f"{station.station_id}    {station.station_name}", fontsize = 30)
        station_counter = 0
        #plt.ylim(0,5)
        for i in range(len(station.ride_request_list_whole)):
            if (station.ready_bike_count_list[i] < (station.ride_request_list_whole)[i]):
                station_counter +=1
                global_counter +=1
                plt.scatter(i,(station.ride_request_list_whole+station.ride_request_list)[i], s = 250,marker = 'X', color ='r',linewidth = 1)
        plt.ylim(1,35)
        if compare_bool:
            plt.plot(compare)
            plt.plot(compare_)
        plt.plot(station.ride_request_list_whole, label = 'Ride Request')
        plt.plot(station.ready_bike_count_list, label = 'Ready bike')
        plt.legend(fontsize=20,loc ="upper right")

        plt.show()
        print (f"No of Failure points in {station.station_id}: {station_counter} ")
    print (f"No of Failure points in overall: {global_counter} ")

def cost_cross_check():
    battery_remaining = 0
    for bike in coimbatore.running_bikes:
        battery_remaining+=(bike.battery_percentage)
    mu_ps_mp = print_station_one_day_report(only_value = True)
    Expense = (mu_ps_mp * no_of_stations) - (battery_remaining*0.27)
    Income = sum(coimbatore.kallapetti)
    print(f"Expense: {Expense}")
    print(f"Income {Income}")
    if (Income>Expense):print(":-)")
    else: print (":-(")
    print()
    battery_consumed = 0
    for book in coimbatore.whole_booking_log:
        battery_consumed+=((book['Battery_depleted']))
    print("Battery consumed: ", battery_consumed)
    battery_remaining = 0
    for bike in coimbatore.running_bikes:
        battery_remaining+=(bike.battery_percentage)
    print(f"Battery Remaining: {battery_remaining}")
    print(f"battery_consumed+battery_remaining: {battery_consumed+battery_remaining}")
    print()
    charging_time = 0
    for bike in coimbatore.running_bikes:
        charging_time+=bike.charging_time
    print("Percentage charged",charging_time*0.27777777778)
    print()
    km_sum = 0
    for book in coimbatore.whole_booking_log:
        km_sum+=book['Requested_km']
    #print("Cost:", charging_time*0.002777777777777778*100*0.27)
    print("Cost based on battery consumed: ", battery_consumed * 0.27)
    print("Cost based on km: ", km_sum*0.3375)
    print("\nBattery:Km ratio :",battery_consumed/km_sum)
    positive = []
    negative = []
    for bikes in coimbatore.running_bikes:
        for b in bikes.sai_list:
            if(b < 0):
                negative.append(b)
            else:
                positive.append(b)
    print("Total Percentage charged: ",sum(positive))
    print("Total Percentage Depleted:",sum(negative))
    Usetime =0
    for book in coimbatore.whole_booking_log:
        Usetime+=book['Usetime']
    print()
    print("Compare Minutes charged: ",len(positive),charging_time)
    print("Compare Minutes depleted:",len(negative),Usetime)
    print()
    print("Battery remaining: ",battery_remaining*0.27)
def plot_all_rides_of_the_day():
    plt.figure(figsize=(15, 15), dpi=100)
    for b in range(len(coimbatore.running_bikes)):
        bike = coimbatore.running_bikes[b]
        for events in range(len(bike.station_history_obj)):
            plt.plot((bike.station_history_obj[events][0].latitude,bike.station_history_obj[events][1].latitude),(bike.station_history_obj[events][0].longitude,bike.station_history_obj[events][1].longitude))


    for station in coimbatore.established_stations:
        plt.scatter(station.latitude,station.longitude)
        plt.text(station.latitude , station.longitude  , station.station_name, fontsize=20)
    plt.axis('off')
    plt.show()
def plot_movie(data,title = "No Name"):
    station_id = [(station.station_id)+"-"+(station.station_name) for station in coimbatore.established_stations]


    for t in range(len(data)):
        Time =Time_con(t)
        #if (t == 200):break
        img = np.zeros((800, 800, 3), np.uint8)
        img[:] = (0, 0, 0)
        y = data[t]
        y = [500 - (i*10) for i in y]

        data_i = [((i*36)+36,y[i]) for i in range(20)]

        #img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        for c,point in enumerate(reversed(data_i)):
            cv2.putText(img, station_id[c], (520,point[0]+36+12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
            #img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        for c,point in enumerate(data_i):
            #cv2.circle(img, tuple(point), 5, (0, 0, 255), -1)
                        #if (int((500 - point[1])/10)>15):co = (0,0,255)
            #elif (int((500 - point[1])/10)<6):co = (255,0,0)
            #else:co = (0,255,0)
            if outward[coimbatore.established_stations[c].station_id][t//60] == "High" : co =  (0,0,255)
            if outward[coimbatore.established_stations[c].station_id][t//60] == "Low" : co =  (255,0,0)

            if ((int((500 - point[1])/10))!=0):
                cv2.line(img,tuple(point),(point[0],500),co,25)
                cv2.putText(img, str(int((500 - point[1])/10)), tuple([(point[0]-10),(point[1]-15)]), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1)

            #cv2.line(img,(20,0),(20,500),(0,255,0),5)
            cv2.putText(img, Time, (500,150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 3)
            cv2.putText(img, title, (100,150), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 1)
            #cv2.putText(img, station_id[c], (point[0],600), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 4)
            #img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        cv2.imshow('Scatter Plot', img)
        cv2.waitKey(100)
    cv2.destroyAllWindows()
#Matplotlib movie
"""def plot_movie(data, title = "No Name"):
    station_id = [(station.station_id)+" - "+(station.station_name) for station in coimbatore.established_stations]
    try:
        start = float(input("Enter Starting time"))
        end = float(input("Enter ending time"))
        inc = int(input ("Enter time interval"))
    except:
        start = 8.00
        end = 23.00
        inc = 60
    start = (int(start) - operation_start_time)*60 + (round((start - int(start))*100))
    end = (int(end) - operation_start_time)*60 + (round((end - int(end))*100))
    for time in range(start,end,inc):
        Time = f"{(int(time)//60)+operation_start_time :02d}:{int(time)%60 :02d} "
        plt.figure(figsize=(15, 8))
        plt.title(title+"\n"+Time)
        plt.bar(np.linspace(0,19,20),np.array(data)[time])
        plt.xticks(np.linspace(0,19,20),station_id,rotation = 90)
        maxv = np.max(data)
        plt.yticks([i for i in range(1,maxv+1)],[i for i in range(1,maxv+1)])
        plt.show()
        cl(wait = True)

"""
def run_analyser(battery_plot,ready_state,max_vehicle,in_ride_bikes,losing_customer,gaining_customer,wingman_accumulation):
    res = input ("Vehicle day Report ?")
    if res == '':print_day_vehicle_report()# vehicle day report
    res = input ("Station day report ?")
    if res == '':print_station_one_day_report() # station day report
    res = input ("Plot Battery plot ?")
    if res == '':plot_report(battery_plot) # Battery of all vehicles over time
    res = input ("Plot Available Bike ?")
    if res == '':plot_available_plot(ready_state) # bar graph ready vehicles in each stations
    #plot_active_vehicle_in_each_station(ready_state) # time series graph ready vehicles in each stations
    res = input ("Print Vehicle Accumulation ?")
    if res == '':plot_vehicles_accumulation(max_vehicle) # vehicle accumulation at each station report
    res = input ("Plot Vehicle Accumulation ?")
    if res == '':vehicle_accumulation()
    #plot_booking_status(book_counter) #no of book for every hour
    res = input ("Plot in ride vehicles ?")
    if res == '':plot_in_ride_status(in_ride_bikes)# average active vehicle for every hour
    res = input ("Plot Customer lose ?")
    if res == '':plot_customer_lose(losing_customer)# losing customer plot
    res = input ("Plot Customer gain ?")
    if res == '':plot_customer_gain(gaining_customer)# almost equal to booking
    res = input ("Plot lose data report ?")
    if res == '':
        if shuffling:
            print_lose_data_report()#customer loss analysis
    res = input ("Wingman Activity ?")
    if res == '':wing_man_activity()
    res = input ("Call stat?")
    if res == '':shuffle_call(log = False)
    res = input ("Plot ready vs request plot ?")
    if res == '':ready_vs_request_plot()#ready_bikes vs request plot - failure time points
    res = input ("Cost analysis ?")
    if res == '':cost_cross_check()
    res = input ("Plot Map ?")
    if res == '':plot_all_rides_of_the_day()
    res = input ("Play Vehicle Accumulation ?")
    if res == '':plot_movie(max_vehicle,"Vehicle Accumulation")
    #cl(wait = True)
    res = input ("Play Losing customer ?")
    if res == '':plot_movie(losing_customer, "Losing Customer")
    #cl(wait = True)
    res = input ("Play Gaining Customer ?")
    if res == '':plot_movie(gaining_customer, "Gaining Customer")
    #cl(wait = True)
    res = input ("Play Wingman Accumulation ?")
    if res == '':plot_movie(wingman_accumulation, "Wingman accumulation")



def operation_monitor_beta(Customer = True, Shuffler = True,wingman_ = True, only_one_station=False,only_one_bike = False):
    size = 700
    pad = 100

    bike_number =""
    station_id = ""
    if only_one_bike:
        bike_number = only_one_bike
        only_one_bike = True
    if only_one_station:
        for st in coimbatore.established_stations:
            if st.station_id == only_one_station:
                station_id_in = st
                only_one_station = True
    # Helper functions___________________________________________________________
    def find_point(orgin,dest,completed_travel):
        x1, y1 = orgin[0], orgin[1]
        x2, y2 = dest[0], dest[1]
        distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        part_dist = completed_travel * distance
        direction = math.atan2(y2 - y1, x2 - x1)
        lat = x1 + part_dist * math.cos(direction)
        lon = y1 + part_dist * math.sin(direction)
        return lat,lon
    def norm_lat(x):
        return((x - min([station.latitude for station in coimbatore.established_stations ]))/(max([station.latitude for station in coimbatore.established_stations ]) - min([station.latitude for station in coimbatore.established_stations ]))*(size/1.2))
    def norm_lon(x):
        return((x - min([station.longitude for station in coimbatore.established_stations ]))/(max([station.longitude for station in coimbatore.established_stations ]) - min([station.longitude for station in coimbatore.established_stations ]))*(size/1.2))

    b = 0
    e =0
    # Dictionary creation__________________________________________________________________________________________
    bikes = dict()
    for bike in coimbatore.running_bikes:
        bikes.update({
        bike.bike_id:{"Id": bike.bike_id,"Running": False,"start": 0 ,"end":0,"latitude": 0,"longitude":0,"user":"",
                        "completed_percentage":0,"Travel_time":0,"Orgin_id": "","dest_id": "","orgin_cor": (),"dest_cor":()}})

    wingman = dict()
    for wm in coimbatore.Wingman:
        wingman.update({wm.employee_id:{ "Latitude": 0, "Longitude": 0, "completed_travel": 0 ,"start_time": 0, "Ending_time": 0, "called": False,"Travel_time": 0,"orgin":"","dest": "" }})
    timestamp = np.linspace(0,((operation_end_time- operation_start_time)*60)-1,(operation_end_time- operation_start_time)*60)
    #Time loop starts here________________________________________________________________________________________
    for time in timestamp:

        Time = f"{(int(time)//60)+operation_start_time :02d}:{int(time)%60 :02d}"
        data6=[]
        for i_d in wingman:
            if wingman[i_d]["called"]:
                wingman[i_d]["Travel_time"]+=1
                wingman[i_d]["Completed_travel"] =  wingman[i_d]["Travel_time"]/( wingman[i_d]["Ending_time"] -  wingman[i_d]["start_time"])
                orgin = (wingman[i_d]["orgin"].latitude,wingman[i_d]["orgin"].longitude)
                dest = (wingman[i_d]["dest"].latitude,wingman[i_d]["dest"].longitude)
                wingman[i_d]["Latitude"],wingman[i_d]["Longitude"] = find_point(orgin,dest,wingman[i_d]["Completed_travel"])
                if only_one_station and (wingman[i_d]["orgin"].station_id == station_id_in.station_id or wingman[i_d]["dest"].station_id == station_id_in.station_id):
                    data6.append([int(norm_lat(wingman[i_d]["Latitude"])),int(norm_lon(wingman[i_d]["Longitude"]))])

                if not only_one_station:
                    data6.append([int(norm_lat(wingman[i_d]["Latitude"])),int(norm_lon(wingman[i_d]["Longitude"]))])
        for key in coimbatore.wingman_calling_log:
            log = coimbatore.wingman_calling_log[key]

            if log["station objects"][1] == time:

                wingman[log["Wingman_id"]]["Latitude"] = log["station objects"][0].latitude
                wingman[log["Wingman_id"]]["Longitude"] = log["station objects"][0].longitude
                wingman[log["Wingman_id"]]["completed_travel"] = 0
                wingman[log["Wingman_id"]]["start_time"] = log["station objects"][1]
                wingman[log["Wingman_id"]]["Ending_time"] = log["station objects"][3]
                wingman[log["Wingman_id"]]["called"] = True
                wingman[log["Wingman_id"]]["Travel_time"] = 0
                wingman[log["Wingman_id"]]["orgin"] = log["station objects"][0]
                wingman[log["Wingman_id"]]["dest"] = log["station objects"][2]


            if log["station objects"][3] == time:
                wingman[log["Wingman_id"]]["Latitude"] = log["station objects"][2].latitude
                wingman[log["Wingman_id"]]["Longitude"] = log["station objects"][2].longitude
                wingman[log["Wingman_id"]]["completed_travel"] = 0
                wingman[log["Wingman_id"]]["start_time"] = 0
                wingman[log["Wingman_id"]]["Ending_time"] = 0
                wingman[log["Wingman_id"]]["called"] = False
                wingman[log["Wingman_id"]]["Travel_time"] = 0
                wingman[log["Wingman_id"]]["orgin"] = log["station objects"][0]
                wingman[log["Wingman_id"]]["dest"] = log["station objects"][2]

        #Update-------------------------------------------------------------------------------------------------------------------
        for book in coimbatore.whole_booking_log:
            pass
            #ride initiate
            if book['Booking_Time'][-6:-1] == Time:
                b+=1
                bikes[book["Bike"]]["Running"]=True
                bikes[book["Bike"]]["user"] = book["User"]
                bikes[book["Bike"]]["completed_percentage"] = 0
                bikes[book["Bike"]]["Orgin_id"] = coimbatore.established_stations[int(book["Orgin"][5:7])-1]
                bikes[book["Bike"]]["dest_id"] = coimbatore.established_stations[int(book["Destination"][5:7])-1]
                bikes[book["Bike"]]["orgin_cor"] = (coimbatore.established_stations[int(book["Orgin"][5:7])-1].latitude,coimbatore.established_stations[int(book["Orgin"][5:7])-1].longitude)
                bikes[book["Bike"]]["dest_cor"] = (coimbatore.established_stations[int(book["Destination"][5:7])-1].latitude,coimbatore.established_stations[int(book["Destination"][5:7])-1].longitude)
                bikes[book["Bike"]]["start"] = (int(book['Booking_Time'][-6:-1][:2]) - operation_start_time)*60 + int(book['Booking_Time'][-6:-1][-2:])
                bikes[book["Bike"]]["end"] = (int(book['Ending_Time'][:-1][:2]) - operation_start_time)*60 + int(book['Ending_Time'][:-1][-2:])
                bikes[book["Bike"]]["Travel_time"] = 0
            if book['Ending_Time'][:-1] == Time:
                e+=1
                bikes[book["Bike"]]["Running"]=False
                bikes[book["Bike"]]["User"] = ""
                bikes[book["Bike"]]["completed_percentage"] = 1
                bikes[book["Bike"]]["Orgin_id"] = ""#coimbatore.established_stations[int(coimbatore.whole_booking_log[0]["Orgin"][5:7])-1]
                bikes[book["Bike"]]["dest_id"] = ""#coimbatore.established_stations[int(coimbatore.whole_booking_log[0]["Destination"][5:7])-1]
                bikes[book["Bike"]]["orgin_cor"] = (0,0)#(coimbatore.established_stations[int(coimbatore.whole_booking_log[0]["Orgin"][5:7])-1].latitude,coimbatore.established_stations[int(coimbatore.whole_booking_log[0]["Orgin"][5:7])-1].longitude)
                bikes[book["Bike"]]["dest_cor"] = (0,0)#(coimbatore.established_stations[int(coimbatore.whole_booking_log[0]["Destination"][5:7])-1].longitude,coimbatore.established_stations[int(coimbatore.whole_booking_log[0]["Destination"][5:7])-1].latitude)
                bikes[book["Bike"]]["start"] =0
                bikes[book["Bike"]]["end"] =0
                bikes[book["Bike"]]["Travel_time"] = 0

        #update
        for i in bikes:
            if(bikes[i]["Running"]):
                bikes[i]["Travel_time"]+=1
                bikes[i]["completed_percentage"]= bikes[i]["Travel_time"]/(bikes[i]["end"] - bikes[i]["start"])
                bikes[i]["latitude"],bikes[i]["longitude"] = find_point (bikes[i]["orgin_cor"],bikes[i]["dest_cor"],bikes[i]["completed_percentage"])

    # CV2__________________________________________________________________________________________________________________
        img = np.zeros((size, size, 3), np.uint8)
        img[:] = (0, 0, 0)
        data = []
        st_name =[]

        for station in coimbatore.established_stations:
            data.append((norm_lat(station.latitude),norm_lon(station.longitude)))
            st_name.append(station.station_name)
        data = np.array(data,dtype = np.int64)
        data+=pad
        cv2.putText(img, Time, (int(0.7*size),int(0.1*size)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 4)
        cv2.putText(img, "Coimbatore City", (int(0.6*size),int(0.05*size)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4)

        for c,point in enumerate(data):
            cv2.circle(img, tuple(point), 10, (255, 0, 0), -1)

            cv2.putText(img, st_name[c], tuple(point), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        data2 =[]
        data3 =[]
        data4 = []
        data5 = []

        data7 = []
        data8 = []
        data9 =[]
        data10 = []
        for i in bikes:
            if (not only_one_bike or i == bike_number):
                if bikes[i]["Running"] and bikes[i]["user"] == "Customer":
                    data2.append((norm_lat(bikes[i]["latitude"]),norm_lon(bikes[i]["longitude"])))
                    data4.append((((norm_lat(bikes[i]["latitude"]),norm_lon(bikes[i]["longitude"]))),(norm_lat(bikes[i]["orgin_cor"][0]),norm_lon(bikes[i]["orgin_cor"][1]))))
                elif bikes[i]["Running"]:
                    data3.append((norm_lat(bikes[i]["latitude"]),norm_lon(bikes[i]["longitude"])))
                    data5.append((((norm_lat(bikes[i]["latitude"]),norm_lon(bikes[i]["longitude"]))),(norm_lat(bikes[i]["orgin_cor"][0]),norm_lon(bikes[i]["orgin_cor"][1]))))

            #print(bikes[i]["Orgin_id"])
            if (not only_one_station or bikes[i]["Orgin_id"] == station_id_in or bikes[i]["dest_id"] == station_id_in):
                if bikes[i]["Running"] and bikes[i]["user"] == "Customer":
                    data7.append((norm_lat(bikes[i]["latitude"]),norm_lon(bikes[i]["longitude"])))
                    data9.append((((norm_lat(bikes[i]["latitude"]),norm_lon(bikes[i]["longitude"]))),(norm_lat(bikes[i]["orgin_cor"][0]),norm_lon(bikes[i]["orgin_cor"][1]))))
                elif bikes[i]["Running"]:
                    data8.append((norm_lat(bikes[i]["latitude"]),norm_lon(bikes[i]["longitude"])))
                    data10.append((((norm_lat(bikes[i]["latitude"]),norm_lon(bikes[i]["longitude"]))),(norm_lat(bikes[i]["orgin_cor"][0]),norm_lon(bikes[i]["orgin_cor"][1]))))

        data2 = np.array(data2,dtype = np.int64)
        data3 = np.array(data3,dtype = np.int64)
        data4 = np.array(data4,dtype = np.int64)
        data5 = np.array(data5,dtype = np.int64)
        data6 = np.array(data6,dtype = np.int64)
        data7 = np.array(data7,dtype = np.int64)
        data8 = np.array(data8,dtype = np.int64)
        data9 = np.array(data9,dtype = np.int64)
        data10 = np.array(data10,dtype = np.int64)
        data2+=pad
        data3+=pad
        data4+=pad
        data5+=pad
        data6+=pad
        data7+=pad
        data8+=pad
        data9+=pad
        data10+=pad
        if not only_one_station:
            for point2 in data2:
                pass
                #print("point2",point2,len(data2),data2)
                if Customer: cv2.circle(img, tuple(point2), 2, (0, 255, 0), -1)
            for point3 in data3:
                if Shuffler: cv2.circle(img, tuple(point3), 2, (0, 0, 255), -1)
            for point4 in data4:
                p1 = tuple(point4[0])
                p2 = tuple(point4[1])
                if Customer: cv2.line(img,p1,p2,(0,255,0),1)
            for point5 in data5:
                p1 = tuple(point5[0])
                p2 = tuple(point5[1])
                if Shuffler: cv2.line(img,p1,p2,(0,0,255),1)

            for point6 in data6:
                if wingman_:cv2.circle(img, tuple(point6), 3, (0, 255, 255), -1)


        if only_one_station:
            for point2 in data7:
                pass

                if Customer: cv2.circle(img, tuple(point2), 2, (0, 255, 0), -1)
            for point3 in data8:
                if Shuffler: cv2.circle(img, tuple(point3), 2, (0, 0, 255), -1)
            for point4 in data9:
                p1 = tuple(point4[0])
                p2 = tuple(point4[1])
                if Customer: cv2.line(img,p1,p2,(0,255,0),1)
            for point5 in data10:
                p1 = tuple(point5[0])
                p2 = tuple(point5[1])
                if Shuffler: cv2.line(img,p1,p2,(0,0,255),1)
            for point6 in data6:
                if wingman_:cv2.circle(img, tuple(point6), 3, (0, 255, 255), -1)
            cv2.putText(img, Time, (int(0.7*size),int(0.1*size)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 4)
            cv2.putText(img, "Coimbatore City", (int(0.6*size),int(0.05*size)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4)
            if only_one_station:

                cv2.putText(img, "Only "+station_id_in.station_name, (int(0.6*size),int(0.15*size)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)
        cv2.imshow('Map', img)
        cv2.waitKey(50)
    cv2.destroyAllWindows()

def vehicle_accumulation():
    fig = plt.figure(figsize=(18, 3))
    for i in range(20):
        max_vehicle = coimbatore.max_vehicle


        plt.plot(np.array(max_vehicle).T[i])
    arr = np.array(coimbatore.max_vehicle)
    max_index = arr.argmax()
    max_row_index, max_col_index = np.unravel_index(max_index, arr.shape)
    print(f"Maximum of {arr[max_row_index, max_col_index]} vehicles accumulated  at {coimbatore.established_stations[max_col_index].station_name} at {(max_row_index//60 + operation_start_time)}:{ (max_row_index%60)}")

    plt.show()
    res = input ("Should I plot all the station? ")
    if res != "":
        for i in range(20):
            max_vehicle = coimbatore.max_vehicle
            fig = plt.figure(figsize=(18, 3))
            plt.plot(np.array(max_vehicle).T[i])
            plt.text(700, np.mean(np.array(max_vehicle).T[i]), f"Maximum: {max(np.array(max_vehicle).T[i])}", fontsize=12, color='red')
            plt.title(f"{coimbatore.established_stations[i].station_name}")
            plt.show()


def wing_man_activity():
    hour_sum =[]
    hour_min_h_sum =[]
    hour_max_h_sum = []
    x = [i+operation_start_time for i in range(operation_end_time - operation_start_time)]
    for i in range(operation_end_time - operation_start_time):
        hour_sum.append(sum(coimbatore.shuffle_logger[i*60:(i+1)*60]))
        hour_min_h_sum.append(sum(coimbatore.min_h_shuffle_logger[i*60:(i+1)*60]))
        hour_max_h_sum.append(sum(coimbatore.max_h_shuffle_logger[i*60:(i+1)*60]))
    fig, ax = plt.subplots()
    plt.title("Wingman Activity")
    ax.bar(x, hour_min_h_sum, label='Min_h')
    ax.bar(x, hour_max_h_sum, bottom=hour_min_h_sum, label='Max_h')
    ax.set_ylim(0,max(hour_sum)+5)
    ax.legend()
    for i in range(len(hour_sum)):
        if hour_sum[i]!= 0 :ax.text(x[i]-0.30,hour_sum[i]+0.5,str(hour_sum[i]))
        if hour_max_h_sum[i]!= 0 :ax.text(x[i]-0.30,hour_min_h_sum[i]/2,str(hour_min_h_sum[i]))
        if hour_max_h_sum[i]!= 0 :ax.text(x[i]-0.25,(hour_max_h_sum[i]/2)+hour_min_h_sum[i],str(hour_max_h_sum[i]))
    ax.set_xticks(x,x)
    ax.set_xlabel("Time")
    ax.set_ylabel("No. of shuffling in 1 hour")
    plt.show()

def shuffle_call(log = False):
    print("No. of call events: ", coimbatore.call_counter)
    print("No. of shuffling: ",coimbatore.shuffle_counter)
    print(f"Wingman ID    |   Km by Call  |  Km by Shuffle |")
    print("________________________________________________")
    for wm in coimbatore.Wingman:
        print(f"{wm.employee_id}    |      {int(wm.call_km)}      |       {int(wm.km)}       |")

    if log:
        for k in range(len(coimbatore.wingman_calling_log)):
            print(f"Call ID : Call{k:03d}")
            for key,element in coimbatore.wingman_calling_log[f"Call{k:03d}"].items():
                if key != "station objects":
                    pass
                    print(key," : ", element)
            print('_____________________________________________________________________________')

### One day operation

In [ ]:
def simulate_for_a_day_(day = 1):
    coimbatore.Day = day
    #print(Day)
    max_shuf_count = 0
    min_shuf_count = 0
    battery_plot = []
    active_plot = []
    ready_state = []
    max_vehicle = []
    wingman_accumulation = []
    wingman_fail =[]
    bike_fail = []
    coimbatore.bike_shuffle_failure_counter =0
    coimbatore.wingman_shuffle_failure_counter = 0
    coimbatore.min_h_shuffle_logger =[]
    coimbatore.max_h_shuffle_logger = []
    coimbatore.wingman_calling_log = dict()
    coimbatore.user_overall_time_counter = []
    coimbatore.total_kms_user_ridden_this_day = 0
    coimbatore.total_shuffle_kms_ridden_this_day = 0
    coimbatore.total_idle_time_this_day = 0
    coimbatore.user_km_counter = []
    coimbatore.shuffle_km_counter = []
    coimbatore.user_idle_time_counter = []


#Creating timestamp to loop through
    timestamp = np.linspace(0,((operation_end_time- operation_start_time)*60)-1,(operation_end_time- operation_start_time)*60)

#Peak time defined here
    peaktimer = []
    for p in range(len(peak_time)):
        peaktimer+=list(np.linspace(int((peak_time[p][0]- operation_start_time)*60),int((peak_time[p][1]- operation_start_time)*60 - 1),(int((peak_time[p][1]- operation_start_time)*60 - 1) - int((peak_time[p][0]- operation_start_time)*60 )+1)))

#Vehicles are charged every night
    coimbatore.over_night_partial_charge()

    coimbatore.ride_counter =0
    coimbatore.shuffle_counter = 0
    booking_counter = []# same as ride_counter
    in_ride_bikes = []
    losing_customer = []
    gaining_customer =[]
    customer_request__ =[]
    mean_ = []
    act = 0
    t_check = 0
    hour_counter = -1
    p_new = dict()
    pbar = ''

    for station in coimbatore.established_stations:
        p_new.update({station.station_id: 0})
    #TEST PRINT. Uncomment below lines to troubleshoot "Engage Recovery status" of each station
    #coimbatore.test_wm =[]

#Time Loop starts here-------------------------------------------------------------------------------------------------------------
    for time in timestamp:

        Time = Time_con(time)
        # NEW Progress bar___________________________________________
        pb = '▐'
        t_pb = int(((time+1)/len(timestamp))*100)
        pbar = (pb*t_pb)
        time_pb = Time_con(time)
        print(time_pb +" " + pbar +" " + str(t_pb) + "%", end='\r')
        #____________________________________________________________

        if time in peaktimer:
            Peaktime = True
        else:
            Peaktime = False
        coimbatore.clock = time


        # OLD Progress bar

        #i = (int(((time+1)/len(timestamp))*100));a = str(Time)+'['+'='*i +'>'+' '*(100-i)+']'+str(i)+' %';print('\r', a, end = "")
        coimbatore.shuffle_logger.append(coimbatore.shuffle_counter - sum(coimbatore.shuffle_logger))
        coimbatore.min_h_shuffle_logger.append(min_shuf_count - sum(coimbatore.min_h_shuffle_logger))
        coimbatore.max_h_shuffle_logger.append(max_shuf_count - sum(coimbatore.max_h_shuffle_logger))

#To update min_h and max_h of each station for each minute
        coimbatore.traffic_mean = []
        coimbatore.shuffle_requesting_stations = []
        coimbatore.shuffle_ready_stations = []
        for station in coimbatore.established_stations:
            coimbatore.traffic_mean.append(outward[station.station_id][int(coimbatore.clock//60)])
        coimbatore.traffic_mean = np.mean(coimbatore.traffic_mean)
        for station in coimbatore.established_stations:
            station.ride_request_list = []
            station.ride_request_list_whole = []
            station.ride_request_list_mean = []
            station.ready_bike_count_list = []
            #station.current_available_wingman = []
            station.shuffle_ready_bikes = []
            #if (time%60 == 0):
            outward_status = outward[station.station_id][int(coimbatore.clock//60)]
            station.outward_traffic_status = outward_status
            if station.outward_traffic_status > coimbatore.traffic_mean:

                station.min_h = min_h_high
                station.max_h = max_h_high
            elif station.outward_traffic_status < coimbatore.traffic_mean:
                station.min_h = min_h_low
                station.max_h = max_h_low


            inward_status = inward[station.station_id][int(coimbatore.clock//60)]
            station.inward_traffic_status = inward_status


            #To update the Ready Bike List of each station
            station.ready_bike_list = []
            station.ready_bike_count = 0
            for bike in station.current_available_bike:
                if not bike.active and bike.battery_percentage > 25:
                    station.ready_bike_list.append(bike)
                    station.ready_bike_count+=1

            #TEST PRINT. Uncomment the below lines to troubleshoot "ready_bike_list" of every station
            '''a = []
            for i in station.ready_bike_list:
                a.append(i.bike_id)
            print(station.station_id, a)'''

            station.shuffle_index = station.ready_bike_count/station.outward_traffic_status
            station.ready_bike_count_list.append(station.ready_bike_count)

            #TEST PRINT. Uncomment below lines to troubleshoot count of "ready_bike_list" of each station
            #print(len(station.ready_bike_list))
            #print(sum(station.ready_bike_count_list))

            #To update the SReqS and SRS in each timestep
                #SReqS => Shuffle Requesting stations
                #SRS => Shuffle Ready Stations

            if (station.ready_bike_count<station.min_h):
                coimbatore.shuffle_requesting_stations.append(station)
            elif(station.ready_bike_count>station.max_h):
                coimbatore.shuffle_ready_stations.append(station)



            weight_list = []
            for station_ in coimbatore.established_stations:
                if Time_chart[int(time//60)][station.station_index][station_.station_index] != 0:
                    w = (station_.ready_bike_count)/((station_.outward_traffic_status)*((Time_chart[int(45//60)][station.station_index][station_.station_index])))
                else: w=0
                weight_list.append(w)
            list1 = [station for station in coimbatore.established_stations]
            zipped = zip(list1, weight_list)
            sorted_tuples = sorted(zipped, key=lambda x: -x[1])
            station.sorted_arriving_stations,_ = zip(*sorted_tuples)

        #TEST PRINT. Uncomment below lines to troubleshoot SReqS and SRS in each timestep

        #print()
        #print("SReqS => ", len(coimbatore.shuffle_requesting_stations))
        #print()
        #print("SRS => ", len(coimbatore.shuffle_ready_stations))
        #print()

#To update the list of Shuffle Ready Bikes in every station in each timestep

        for station in coimbatore.shuffle_ready_stations:
            s_check = 0
            station.shuffle_ready_bikes = []
            for bike in station.ready_bike_list:
                if(s_check < (station.ready_bike_count-station.max_h)):
                    station.shuffle_ready_bikes.append(bike)
                    s_check+=1

        #TEST PRINT. Uncomment the below lines to troubleshoot "shuffle_ready_bikes" of every station
        '''
        for station in coimbatore.established_stations:
            a = []
            for b in station.shuffle_ready_bikes:
                a.append(b.bike_id)
            print(station.station_id, ' => ', a)'''

            #TEST PRINT. Uncomment below line to troubleshoot count of "shuffle_ready_bikes" of each station
            #print(s_check, len(station.shuffle_ready_bikes))

            #TEST PRINT. Uncomment below lines to troubleshoot "bikes avilable for shuffling" in each station

            #if station.ready_bike_count >= station.max_h:
                #print(station.station_id, " => ", (station.ready_bike_count-station.max_h))



#To update the travel time chart for each station for each hour

        if (time%60 == 0 and time<840):

            #if time!=0:
            hour_counter+=1
            for j, station in enumerate(coimbatore.established_stations):
                station.travel_time_chart = []
                time_temp_list = []
                for i in Time_chart[hour_counter][j]:
                    time_temp_list.append(i)
                    station.travel_time_chart.append((time_temp_list.index(i), i, len(coimbatore.established_stations[time_temp_list.index(i)].shuffle_ready_bikes),coimbatore.established_stations[time_temp_list.index(i)]))


                station.sorted_travel_time_chart = sorted(station.travel_time_chart, key=lambda x: x[1])
                station.sorted_travel_time_chart.pop(j)
                #TEST PRINT. Uncomment below line to troubleshoot "travel_time_chart" of each station
                #print(station.travel_time_chart)
                #print(station.sorted_travel_time_chart)



#Recovey time____________________________________________________________________________________________
        #TEST PRINT. Uncomment below lines to troubleshoot "Engage Recovery status" of each station
        #coimbatore.test_wm.append((Time,coimbatore.Wingman[0].engaged,coimbatore.Wingman[0].recovery))
        #Recovery time
        for wm in coimbatore.Wingman:

            if wm.engaged and not wm.available and wm.eta<=time:
                coimbatore.close_wingman_call(wm)

        station_obj = [station for station in coimbatore.established_stations]
        #for station in coimbatore.established_stations:

        shuffle_index_list = [station.shuffle_index for station in coimbatore.established_stations ]
        coimbatore.sorted_deficit_station = sorted(station_obj, key=lambda x: shuffle_index_list[station_obj.index(x)])



#SHUFFLING--------------------------------------------------------------------------------------------------------

        if shuffling and (time < ( ( (operation_end_time-8) *60) - shuffle_closing_time)):
            wm_fail_count =0
            bike_fail_count =0
            for station_d in coimbatore.shuffle_requesting_stations:
                sorted_shuffle_bikes = []
                a = []
                '''for station in coimbatore.established_stations:
                    for i in station.ready_bike_list:
                        a.append(i.bike_id)
                    print(station.station_id, a)'''
                for t_value in station_d.sorted_travel_time_chart:
                    sorted_shuffle_bikes+=coimbatore.established_stations[t_value[0]].shuffle_ready_bikes

                #print(len(sorted_shuffle_bikes))
                if (len(sorted_shuffle_bikes) > 0):
                    wingman = False
                    shuffle_bike = False
                    for bike in sorted_shuffle_bikes:
                        if len(bike.station.current_available_wingman) > 0:
                            shuffle_bike = bike
                            wingman = True
                            wingman_obj = shuffle_bike.station.current_available_wingman[0]
                            break

                    if not shuffle_bike:
                        wingman_dest = sorted_shuffle_bikes[0].station
                        coimbatore.call_wingman(wingman_dest, time)

                    if wingman:
                        bike = shuffle_bike
                        origin = bike.station
                        dest = station_d
                        travel_time = int(Time_chart[int(time//60)][int(origin.station_index)][int(dest.station_index)])
                        #print(bike.bike_id)
                        #print(origin.station_id)

                        coimbatore.shuffle_ride_initiate(bike, origin, dest, travel_time, time, day, wingman_obj)
                        min_shuf_count+=1
                        wingman = False
                        coimbatore.ride_counter+=1
                    else:
                        coimbatore.shuffling_failure_counter+=1
                        coimbatore.wingman_shuffle_failure_counter+=1
                        wm_fail_count+=1
                else:
                    coimbatore.shuffling_failure_counter+=1
                    coimbatore.bike_shuffle_failure_counter+=1
                    bike_fail_count+=1


# Max_h_________________________________________________________________________________________________________

#SHUFFLING--------------------------------------------------------------------------------------------------------

        if shuffling and  (time < ( ( (operation_end_time-8) *60) - shuffle_closing_time)):
            shuffle_ready_station_objects =[]
            shuffle_ready_station_count = []
            shuffle_requesting_station_objects = []
            shuffle_requesting_station_count =[]

            #Sort donating station in descending order
            shuffle_ready_station_objects = [station for station in coimbatore.shuffle_ready_stations]
            shuffle_ready_station_count = [station.ready_bike_count for station in coimbatore.shuffle_ready_stations ]
            zipped = zip(shuffle_ready_station_objects, shuffle_ready_station_count)
            sorted_tuples = sorted(zipped, key=lambda x: -x[1])
            try:shuffle_ready_station_objects, shuffle_ready_station_count = zip(*sorted_tuples)
            except:pass
            #Sort requesting station in ascending order
            shuffle_requesting_station_objects = [station for station in coimbatore.shuffle_requesting_stations]
            shuffle_requesting_station_count = [station.ready_bike_count for station in coimbatore.shuffle_requesting_stations ]
            zipped = zip(shuffle_requesting_station_objects, shuffle_requesting_station_count)
            sorted_tuples = sorted(zipped, key=lambda x: x[1])
            try:shuffle_requesting_station_objects, shuffle_requesting_station_count = zip(*sorted_tuples)
            except:pass




            if len(shuffle_requesting_station_objects)>0:
                deficit_station_objects = shuffle_requesting_station_objects
            elif len(shuffle_requesting_station_objects) == 0:
                deficit_station_objects = coimbatore.sorted_deficit_station

            for station_d in deficit_station_objects :
                sorted_shuffle_bikes = []
                a = []
                '''for station in coimbatore.established_stations:
                    for i in station.ready_bike_list:
                        a.append(i.bike_id)
                    print(station.station_id, a)'''
                for station_o in shuffle_ready_station_objects:
                    sorted_shuffle_bikes+=station_o.shuffle_ready_bikes




                #print(len(sorted_shuffle_bikes))
                if (len(sorted_shuffle_bikes) > 0):
                    wingman = False
                    shuffle_bike = sorted_shuffle_bikes[0]
                    if len(sorted_shuffle_bikes[0].station.current_available_wingman)>0:
                        wingman = True
                        wingman_obj = sorted_shuffle_bikes[0].station.current_available_wingman[0]

                    elif (len(shuffle_bike.station.current_available_wingman)==0):
                        wingman_dest = shuffle_bike.station
                        coimbatore.call_wingman(wingman_dest, time)


                    if wingman:
                        bike = shuffle_bike
                        origin = bike.station
                        dest = station_d
                        travel_time = int(Time_chart[int(time//60)][int(origin.station_index)][int(dest.station_index)])
                        #print(bike.bike_id)
                        #print(origin.station_id)
                        coimbatore.shuffle_ride_initiate(bike, origin, dest, travel_time, time, day, wingman_obj)
                        max_shuf_count+=1
                        wingman = False
                        coimbatore.ride_counter+=1
                    else:
                        coimbatore.shuffling_failure_counter+=1
                        coimbatore.wingman_shuffle_failure_counter+=1
                else:
                    coimbatore.shuffling_failure_counter+=1
                    coimbatore.bike_shuffle_failure_counter+=1


        if (time%60 ==0 ):book = 0

        ref1 = []
        ref2 = []
        in_ride_bikes_counter = 0

# CHECKS---------------------------------------------------------------------------------------------------------
# Performing checks on all the vehicles for each timestep

        for bike_i in coimbatore.running_bikes:

            # RIDE_COUNTER Incrementor
            if bike_i.active:
                in_ride_bikes_counter+=1
                bike_i.ready = False

            # BIKE_READY Ensurer
            if not bike_i.active and bike_i.battery_percentage > 25: bike_i.ready = True

            #for Plotting purpose
            ref1.append(bike_i.ready)
            ref2.append(bike_i.battery_percentage)

            # BIKE_USE check (to ensure battery is draining only when the bike is in "use") (due to idle time)
            if(bike_i.km_counter>= bike_i.km):
                bike_i.use = False

            # BIKE_CHARGER
            # oru nimisathiku 0.2% charge yerum
            if bike_i.charging and bike_i.battery_percentage<100 and not bike_i.active:
                bike_i.battery_percentage += 0.27777777778 # 1/(6*60)
                bike_i.sai_list.append(0.27777777778)
                bike_i.charging_time +=1
                bike_i.station.charging_time +=1

            # RUNTIME incrementor
            if bike_i.active: bike_i.running_time+=1

            # BATTERY Depletor
            # deplete only when bike is in the time period of use time
            if bike_i.use:
                bike_i.battery_percentage -=(bike_i.km/bike_i.use_time)*(1.25) # (km/min)*1.25% per km
                bike_i.sai_list.append(-1*((bike_i.km/bike_i.use_time)*(1.25)))
                bike_i.km_counter += (bike_i.km/bike_i.use_time) # increasing the km_counter value by km/min

            # RIDE_ENDER
            # ride aah customer mudichitana nu check pannu
            if ((bike_i.ride_start_time+ bike_i.travel_time) <= time and not bike_i.shuffling) and bike_i.active :
                coimbatore.end_ride(bike_i,day,time)

                #use time end check pannu
                #use time = traveltime - idle time
                #stop using if the km_counter hits the km

            # SHUFFLE_RIDE_TERMINATOR
            if (bike_i.ride_start_time+ bike_i.travel_time <= time and  bike_i.shuffling) and bike_i.active :
                coimbatore.shuffle_ride_terminate(bike_i,day,time)

            # CHARGE_DISCONNECTOR
            # charge full aiydicha nu check pannu
            if (bike_i.charging and bike_i.battery_percentage >= 100):#or((bike_i.charging and time - bike_i.charge_start_time)>(6*60)):
                bike_i.charging = False
                bike_i.charge_start_time = 0

            # CHARGE_CONNECTOR
            # bike aah charge la podu
            if bike_i.charging == False and bike_i.active == False and bike_i.battery_percentage<100:
                bike_i.charge_vehicle(time)

        # TO UPDATE THE LIST OF "in_ride_bikes"
        in_ride_bikes.append(in_ride_bikes_counter)

        #plotting
        active_plot.append(ref1)
        battery_plot.append(ref2)

        ready_state_ =[]

        # bike ready count aah update pannu
        station_max_temp = []
        wingman_accumulation_per_time_step = []
        for station in coimbatore.established_stations:
            wingman_accumulation_per_time_step.append(len(station.current_available_wingman))
            station_max_temp.append(len(station.current_available_bike))

            # Mean of ready_bike_count (not used in sim. as of V.12.03)
            if len(station.ready_bike_count_list)!=0:
                station.ready_bike_count_list_mean.append(np.mean(station.ready_bike_count_list))
            else:
                station.ready_bike_count_list_mean.append(0)

            if (time%60 ==0 ):ready_state_.append(station.ready_bike_count)
        wingman_accumulation.append(wingman_accumulation_per_time_step)
        if (time%60 ==0 ):ready_state.append(ready_state_)

        max_vehicle.append(station_max_temp )

        # render pannu
        if (time%ppts == 0) and plot: print('\n','\n','\n');plot_stat(time)

        losing_customer_per_timestep =[]
        gaining_customer_per_timestep = []
        p = p_new

# BOOKING OF A RIDE----------------------------------------------------------------------------------------------

        act+=1
        p = coimbatore.customer_request()
        customer_request__.append((Time,time,p))

        for count, station in enumerate(coimbatore.established_stations):
            losing_customer_per_station = 0
            gaining_customer_per_station = 0
            request = p[station.station_id] #fetching customer request for each minute in every station
            request_approved  = 0

            for vehicle in station.ready_bike_list:
                if request_approved < request:

                    #To update "traffic_probability" of every station
                    if (time < ( ( (operation_end_time-8) *60) - closing_time)):
                        traffic_probablity = np.zeros(len(coimbatore.established_stations))

                        for count, station_d in enumerate(coimbatore.established_stations):
                            traffic_probablity[count] = station_d.inward_traffic_status

                        #Assigning of "traffic_probability"
                        traffic_probablity/=sum(traffic_probablity)

                        #Assigning of "destination"
                        destination = np.random.choice(coimbatore.established_stations,p = traffic_probablity)

                        #Assigning of "use_time", which is the time for which the user actually rides the bike
                        if(station.station_id == destination.station_id ):
                            km = np.random.randint(2,10)
                            use_time = km *np.random.randint(2,5)
                        else:
                            km = distance_matrix[int(station.station_index)][int(destination.station_index)]
                            use_time = float(Time_chart[int(time//60)][int(station.station_index)][int(destination.station_index)])

                        #Assigning of "idle_time". Time for which user possess the bike but don't use it
                        idle_time = np.random.choice([np.random.randint(0,5),np.random.randint(5,20),np.random.randint(20,120)],p = [0.7,0.2,0.1])

                        #total time at which the bike is not available to other customer or shuffler
                        #time at which the bike is handled by the customer
                        travel_time = use_time + idle_time

                        #passing use time and the kms (used only for simulation not taken into account for booking)

                    # ACTUAL BOOKING-------------------------------------------------------------------------
                        coimbatore.book_ride(vehicle,station,time,travel_time,use_time,km,day,destination)
                    #----------------------------------------------------------------------------------------

                        coimbatore.ride_counter+=1
                        book+=1
                        request_approved +=1
                        gaining_customer_per_station +=1

            if (request_approved < request and (time < (((operation_end_time-8)*60) - closing_time) )):
                losing_customer_per_station = request - request_approved

            losing_customer_per_timestep.append(losing_customer_per_station)
            gaining_customer_per_timestep.append(gaining_customer_per_station)

        if(len(losing_customer_per_timestep)==0):losing_customer_per_timestep= [0]*no_of_stations

        if(len(gaining_customer_per_timestep)==0):gaining_customer_per_timestep= [0]*no_of_stations

        losing_customer.append(losing_customer_per_timestep)
        gaining_customer.append(gaining_customer_per_timestep)

        for station in coimbatore.established_stations:
            station.ride_request_list.append(p[station.station_id])
            station.ride_request_list_whole.append(p[station.station_id])
            station.ride_request_list_mean.append(np.mean(station.ride_request_list))

        if (time%60 ==0 ):booking_counter.append(book)
        bike_fail.append(bike_fail_count)
        wingman_fail.append(wm_fail_count)

#time loop ends________________________________________________________________________________________________________________________
    print("\r"," "*500, end=" ")

    print (f"Total number of rides booked in this day {coimbatore.ride_counter - coimbatore.shuffle_counter} ")
    print(f"Percentage of customer lost in this day {round((np.sum(np.array(losing_customer))/((np.sum(np.array(gaining_customer)))+(np.sum(np.array(losing_customer))))*100),2)} % \n")

    coimbatore.losing_customer_percentage = round((np.sum(np.array(losing_customer))/((np.sum(np.array(gaining_customer)))+(np.sum(np.array(losing_customer))))*100),2)
    #Test Print for troubleshooting Min_h and Max_h shuffling
    #print("min_h: " ,min_shuf_count,"\nMax_h: ", max_shuf_count)
    coimbatore.in_ride_bikes = in_ride_bikes
    coimbatore.bike_fail = bike_fail
    coimbatore.wingman_fail = wingman_fail
    coimbatore.max_vehicle = max_vehicle
    coimbatore.total_kms_user_ridden_this_day = sum(coimbatore.user_km_counter)
    coimbatore.total_kms_user_ridden+=sum(coimbatore.user_km_counter)
    coimbatore.total_shuffle_kms_ridden_this_day = sum(coimbatore.shuffle_km_counter)
    coimbatore.total_shuffle_kms_ridden+=sum(coimbatore.shuffle_km_counter)
    coimbatore.total_idle_time_this_day = sum(coimbatore.user_idle_time_counter)
    coimbatore.total_idle_time+=sum(coimbatore.user_idle_time_counter)

    return ready_state, battery_plot, max_vehicle, booking_counter, in_ride_bikes, losing_customer,gaining_customer,customer_request__,mean_,wingman_accumulation, shuffle_index_list

### One Month Operation

In [ ]:
def simulate_for_a_month():
    coimbatore.reset_day()
    for i in range(no_of_days):
        print ("\n",Fore.BLUE+f"Day {i+1}:"+Fore.RESET)#, end ="")
        simulate_for_a_day_(day = i+1)
        Expense.calculate()
        Accounts.generate_projected_bill()
        Expense.re_calculate()
        #Expense.show_day_expenses()
        Income.calculate()
        #Income.show_income_this_day()
        Accounts.calculate_profit_this_day()
        #Accounts.show_profit_this_day()
        coimbatore.whole_booking_log += coimbatore.booking_log
        coimbatore.booking_log = []
        #Expense.show_expenses_so_far()
        #Income.show_income_so_far()
        Accounts.calculate_profit_so_far()
        #Accounts.show_profit_so_far()
        print(Fore.BLUE+"___________________________________________________________________________________________________________________"+Fore.RESET)
    #Expense.show_expenses_so_far()
    #Income.show_income_so_far()
    Accounts.calculate_profit_so_far()
    #Accounts.show_profit_so_far()

# One day simulation

In [ ]:
coimbatore.whole_booking_log = []
coimbatore.reset_day()
plot = 0
shuffling = 1
ready_state, battery_plot, max_vehicle,book_counter,in_ride_bikes,losing_customer,gaining_customer,customer_request__,mean_,wingman_accumulation,shuffle_index_list = simulate_for_a_day_()
Expense.calculate()
Accounts.generate_projected_bill()
Expense.show_day_expenses()
Income.calculate()
Income.show_income_this_day()
Accounts.calculate_profit_this_day()
Accounts.show_profit_this_day()
coimbatore.whole_booking_log+=coimbatore.booking_log
coimbatore.booking_log = []

### Analyser

In [ ]:
run_analyser(battery_plot,ready_state,max_vehicle,in_ride_bikes,losing_customer,gaining_customer,wingman_accumulation)

### Operation Monitor

In [ ]:
operation_monitor_beta()#(Customer = False,Shuffler = True,wingman_ = False, only_one_station = False , only_one_bike = False)

# One month simulation

In [ ]:
plot = 0
shuffling = 1
simulate_for_a_month()

# Workspace